In [40]:
from AIAnswerEvaluationSystem.neo4j_Manager import Neo4jManager
from AIAnswerEvaluationSystem.logger import logger
import os


NEO4J_URI = os.getenv("NEO4J_URI", "neo4j://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "your_secure_password")


neo4j_manager = Neo4jManager.get_instance(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)


In [41]:
query = """MATCH (sub:Subject)-[:HAS_QUESTION]->(q:Question)
WHERE sub.name = "Artificial Intelligence"
RETURN q.id, q.text, sub.name
ORDER BY q.id;"""

In [42]:
try:
    with neo4j_manager.get_connection().session() as session:
        # Define the query
        query = """
        MATCH (sub:Subject)-[:HAS_QUESTION]->(q:Question)
        WHERE sub.name = $subject_name
        RETURN q.id, q.text, sub.name
        ORDER BY q.id
        """
        
        # Execute the query with parameters
        results = session.execute_read(
            lambda tx: list(tx.run(query, subject_name="Artificial Intelligence"))
        )
        
        if not results:
            logger.info(" No questions found for the specified subject.")
            print(" No questions found for the specified subject.")
        else:
            print(results)
            
except Exception as e:
    logger.error(f" Database error while fetching questions: {str(e)}")

[<Record q.id='AI001_Q1' q.text='Define Artificial Intelligence' sub.name='Artificial Intelligence'>, <Record q.id='AI001_Q2' q.text='What is Machine Learning?' sub.name='Artificial Intelligence'>, <Record q.id='AI001_Q3' q.text='Explain the difference between Supervised and Unsupervised Learning.' sub.name='Artificial Intelligence'>, <Record q.id='AI001_Q4' q.text='What are Neural Networks?' sub.name='Artificial Intelligence'>, <Record q.id='AI001_Q5' q.text='Describe the Turing Test.' sub.name='Artificial Intelligence'>, <Record q.id='AI001_Q6' q.text='What are the main applications of AI?' sub.name='Artificial Intelligence'>]


In [43]:
try:
    with neo4j_manager.get_connection().session() as session:
        query = """
        MATCH (sub:Subject)-[:HAS_QUESTION]->(q:Question)
        WHERE sub.name = $subject_name
        RETURN q.id, q.text, sub.name
        ORDER BY q.id
        """
        
        results = session.execute_read(
            lambda tx: list(tx.run(query, subject_name="Artificial Intelligence"))
        )
        
        if not results:
            logger.info(" No questions found for the specified subject.")
            print(" No questions found for the specified subject.")
        else:
            # Print results line by line
            print("Question ID | Question Text | Subject")
            print("-" * 80)
            for record in results:
                print(f"{record['q.id']} | {record['q.text']} | {record['sub.name']}")
            
except Exception as e:
    logger.error(f" Database error while fetching questions: {str(e)}")

Question ID | Question Text | Subject
--------------------------------------------------------------------------------
AI001_Q1 | Define Artificial Intelligence | Artificial Intelligence
AI001_Q2 | What is Machine Learning? | Artificial Intelligence
AI001_Q3 | Explain the difference between Supervised and Unsupervised Learning. | Artificial Intelligence
AI001_Q4 | What are Neural Networks? | Artificial Intelligence
AI001_Q5 | Describe the Turing Test. | Artificial Intelligence
AI001_Q6 | What are the main applications of AI? | Artificial Intelligence


In [44]:
try:
    with neo4j_manager.get_connection().session() as session:
        query = """MATCH (sub:Subject)-[:HAS_QUESTION]->(q:Question)-[:HAS_ANSWER]->(a:Answer)
        WHERE sub.name = "Artificial Intelligence"
        RETURN sub.name, q.id, q.text, a.text AS answer_text
        ORDER BY q.id;"""

        results = session.execute_read(
            lambda tx: list(tx.run(query, subject_name="Artificial Intelligence"))
        )
        if not results:
            logger.info(" No answers found for the specified subject.")
            print(" No answers found for the specified subject.")

        else:
            print("Subject | Question ID | Question Text | Answer Text")
            print("-" * 80)
            for record in results:
                print(f"{record['sub.name']} | {record['q.id']} | {record['q.text']} | {record['answer_text']}")

except Exception as e:
    logger.error(f" Database error while fetching answers: {str(e)}")

Subject | Question ID | Question Text | Answer Text
--------------------------------------------------------------------------------
Artificial Intelligence | AI001_Q1 | Define Artificial Intelligence | AI simulates human intelligence in machines


In [47]:
try:
    with neo4j_manager.get_connection().session() as session:
        query = """MATCH (s:Student {roll_no: "2115500007"})
                    MATCH (s)-[:SUBMITTED]->(e:ExamSubmission)
                    MATCH (e)-[:FOR_SUBJECT]->(sub:Subject)
                    MATCH (e)-[:HAS_ANSWER]->(a:Answer)
                    WHERE sub.name = "Artificial Intelligence"
                    RETURN 
                        sub.name as subject,
                        a.id as answer_id,
                        a.text as student_answer
                    ORDER BY answer_id"""

        results = session.execute_read(
            lambda tx: list(tx.run(query, subject_name=" Chemistry"))
        )
        if not results:
            logger.info(" No answers found for the specified subject.")
            print(" No answers found for the specified subject.")

        else:
            print("Subject | Answer ID | Answer Text | ")
            print("-" * 80)
            for record in results:
                print(f"{record['subject']} | {record['answer_id']} | {record['student_answer']} ")

except Exception as e:
    logger.error(f" Database error while fetching answers: {str(e)}")

 No answers found for the specified subject.


In [39]:
try:
    with neo4j_manager.get_connection().session() as session:
        query = """// Compare student answers with correct answers
                MATCH (s:Student {roll_no: "2115500007"})
                MATCH (s)-[:SUBMITTED]->(e:ExamSubmission)-[:FOR_SUBJECT]->(sub:Subject {name: "Artificial Intelligence"})
                MATCH (e)-[:HAS_ANSWER]->(sa:Answer)-[:FOR_QUESTION]->(q:Question)
                WITH s, q, sa
                MATCH (sub:Subject)-[:HAS_QUESTION]->(q)-[:HAS_ANSWER]->(ca:Answer)
                WHERE sub.name = "Artificial Intelligence"
                RETURN 
                    s.roll_no as student_roll,
                    s.name as student_name,
                    q.id as question_id,
                    q.text as question_text,
                    sa.text as student_answer,
                    ca.text as correct_answer
                ORDER BY q.id"""
        
        results = session.execute_read(
            lambda tx: list(tx.run(query))
        )

        if not results:
            logger.info("No answers found for the specified subject.")
            print("No answers found for the specified subject.")

        else:
            print("Student Roll No | Student Name| Question ID | Question Text | Student Answer | Correct Answer")
            print("-" * 80)
            for record in results:
                print(f"{record['student_roll']} | {record['student_name']} | {record['question_id']} | {record['question_text']} | {record['student_answer']} | {record['correct_answer']}")
except Exception as e:
    logger.error(f" Database error while fetching answers: {str(e)}")

NameError: name 'logger' is not defined

In [13]:
from typing import Optional
from langchain_ollama import OllamaLLM
from AIAnswerEvaluationSystem.logger import logging

class LLMSS:
    def __init__(self, llm: Optional[OllamaLLM] = None):
        """
        Initialize the LLMSS with an optional LLM.
        
        :param llm: Optional pre-configured OllamaLLM instance
        """
        try:
            self.llm = llm if llm is not None else OllamaLLM(model="llama3.2:latest")
            logging.info("LLMSS initialized successfully")
        except Exception as e:
            logging.error(f"Failed to initialize LLMSS: {e}")
            raise




In [ ]:
from typing import Optional, Dict, List, Union, Tuple
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from AIAnswerEvaluationSystem.logger import logging
import json
import re

class LLMSS:
    def __init__(self, llm: Optional[OllamaLLM] = None):
        """
        Initialize the LLMSS with an optional LLM.
        
        :param llm: Optional pre-configured OllamaLLM instance
        """
        try:
            self.llm = llm if llm is not None else OllamaLLM(model="llama3.2:latest")
            logging.info("LLMSS initialized successfully")
        except Exception as e:
            logging.error(f"Failed to initialize LLMSS: {e}")
            raise


class GradingSystem:
    """Handles converting evaluation scores to letter grades and determining pass/fail status."""
    
    # Grade scale mapping percentage ranges to letter grades
    GRADE_SCALE = {
        (90, 100): 'A',
        (80, 89.99): 'B',
        (70, 79.99): 'C',
        (60, 69.99): 'D',
        (0, 59.99): 'F'
    }
    
    # Minimum passing grade threshold (as a percentage)
    PASSING_THRESHOLD = 60.0
    
    @classmethod
    def calculate_percentage(cls, score: int, max_score: int) -> float:
        """
        Calculate percentage score.
        
        :param score: Achieved score
        :param max_score: Maximum possible score
        :return: Percentage as float
        """
        if max_score <= 0:
            logging.error("Invalid max_score (must be > 0)")
            return 0.0
        
        return (score / max_score) * 100
    
    @classmethod
    def determine_letter_grade(cls, percentage: float) -> str:
        """
        Convert percentage to letter grade.
        
        :param percentage: Score as percentage
        :return: Letter grade (A-F)
        """
        for score_range, grade in cls.GRADE_SCALE.items():
            if score_range[0] <= percentage <= score_range[1]:
                return grade
        
        logging.error(f"Unable to determine grade for percentage: {percentage}")
        return 'N/A'
    
    @classmethod
    def is_passing(cls, percentage: float) -> bool:
        """
        Determine if the score is passing.
        
        :param percentage: Score as percentage
        :return: True if passing, False otherwise
        """
        return percentage >= cls.PASSING_THRESHOLD
    
    @classmethod
    def grade_evaluation(cls, evaluation: Dict, max_score: int = 40) -> Dict:
        """
        Add grading information to an evaluation result.
        
        :param evaluation: Evaluation dictionary with scores
        :param max_score: Maximum possible score (default: 40 based on our scoring criteria)
        :return: Evaluation with added grading information
        """
        try:
            total_score = evaluation.get('total_score', 0)
            percentage = cls.calculate_percentage(total_score, max_score)
            letter_grade = cls.determine_letter_grade(percentage)
            passing = cls.is_passing(percentage)
            
            # Add grading information to the evaluation
            evaluation.update({
                'percentage': round(percentage, 2),
                'letter_grade': letter_grade,
                'passing': passing
            })
            
            return evaluation
        except Exception as e:
            logging.error(f"Error in grading evaluation: {e}")
            # Return original evaluation with error information
            evaluation.update({
                'grading_error': str(e)
            })
            return evaluation


class StudentAnswerEvaluator:
    def __init__(self, llmss: LLMSS):
        """
        Initialize the StudentAnswerEvaluator with an LLMSS instance.
        
        :param llmss: An initialized LLMSS instance
        """
        self.llmss = llmss
        self._setup_evaluation_chain()
        logging.info("StudentAnswerEvaluator initialized successfully")
    
    def _setup_evaluation_chain(self):
        """Set up the evaluation chain with appropriate prompt templates."""
        evaluation_template = """
        You are an expert educator tasked with evaluating student answers.
        
        Question: {question}
        
        Correct Answer: {correct_answer}
        
        Student Answer: {student_answer}
        
        Question Mark Value: {max_marks}
        
        Evaluate the student's answer based on the following criteria:
        1. Correctness (0-10): How factually correct is the answer?
        2. Completeness (0-10): Does the answer cover all required elements?
        3. Clarity (0-10): How clear and well-articulated is the answer?
        4. Critical Thinking (0-10): Does the answer demonstrate understanding and critical thinking?
        
        Provide a total score out of 40, which will be converted to a score out of the maximum marks ({max_marks}).
        
        Also provide:
        - Specific feedback on what was good
        - Specific feedback on what could be improved
        - Suggested correct answer or improvements
        
        Your response MUST be in valid JSON format like this:
        {{
            "correctness_score": 8,
            "completeness_score": 7,
            "clarity_score": 9,
            "critical_thinking_score": 6,
            "total_score": 30,
            "marks_awarded": 15.0,
            "positive_feedback": "The student demonstrated good understanding of...",
            "areas_for_improvement": "The student could improve by...",
            "suggested_improvements": "A more complete answer would include..."
        }}
        
        Do not include any text outside the JSON object. The JSON must be properly formatted with no trailing commas.
        """
        
        self.evaluation_prompt = PromptTemplate(
            input_variables=["question", "correct_answer", "student_answer", "max_marks"],
            template=evaluation_template,
        )
        
        self.evaluation_chain = LLMChain(
            llm=self.llmss.llm,
            prompt=self.evaluation_prompt,
            verbose=False
        )
    
    def _extract_json(self, text: str) -> Dict:
        """
        Extract a valid JSON object from text, with improved error handling.
        
        :param text: Text potentially containing JSON
        :return: Extracted JSON as dictionary or error dict
        """
        # First try direct parsing
        text = text.strip()
        try:
            return json.loads(text)
        except json.JSONDecodeError:
            pass  # Continue to other extraction methods
        
        # Look for JSON with regex - find content between curly braces
        try:
            json_match = re.search(r'({[\s\S]*?})(?:\s*$|\n)', text)
            if json_match:
                json_str = json_match.group(1)
                return json.loads(json_str)
        except (json.JSONDecodeError, AttributeError):
            pass
        
        # Try to fix common JSON formatting issues
        try:
            # Replace single quotes with double quotes
            fixed_text = text.replace("'", '"')
            return json.loads(fixed_text)
        except json.JSONDecodeError:
            pass
        
        # If all extraction methods fail, create a default evaluation
        logging.error(f"All JSON extraction methods failed. Raw text: {text[:100]}...")
        return {
            "error": "Failed to parse result as JSON",
            "raw_output": text
        }
    
    def evaluate_answer(self, question: str, correct_answer: str, student_answer: str, max_marks: float = 10.0) -> Dict:
        """
        Evaluate a student's answer against the correct answer for a given question.
        
        :param question: The question being answered
        :param correct_answer: The correct or model answer
        :param student_answer: The student's submitted answer
        :param max_marks: Maximum marks for this question
        :return: Dictionary containing evaluation scores and feedback
        """
        try:
            logging.info(f"Evaluating answer for question: {question[:50]}...")
            
            # Create a default evaluation with required fields
            default_evaluation = {
                "correctness_score": 0,
                "completeness_score": 0,
                "clarity_score": 0,
                "critical_thinking_score": 0,
                "total_score": 0,
                "marks_awarded": 0.0,
                "positive_feedback": "",
                "areas_for_improvement": "",
                "suggested_improvements": ""
            }
            
            result = self.evaluation_chain.invoke({
                "question": question,
                "correct_answer": correct_answer,
                "student_answer": student_answer,
                "max_marks": max_marks
            })
            
            # Extract JSON from the result with improved handling
            evaluation = self._extract_json(result['text'])
            
            # If there was an error in extraction, return the error info
            if "error" in evaluation:
                return evaluation
            
            # Update default evaluation with extracted values (ensures all keys exist)
            default_evaluation.update(evaluation)
            
            # Calculate marks awarded if not already in the result
            if 'marks_awarded' not in evaluation:
                default_evaluation['marks_awarded'] = (default_evaluation['total_score'] / 40) * max_marks
            
            # Apply grading to the evaluation
            graded_evaluation = GradingSystem.grade_evaluation(default_evaluation)
            
            logging.info(f"Evaluation complete. Total score: {graded_evaluation.get('total_score', 'N/A')}, Grade: {graded_evaluation.get('letter_grade', 'N/A')}")
            return graded_evaluation
        except Exception as e:
            logging.error(f"Error during answer evaluation: {e}")
            return {
                "error": f"Evaluation failed: {str(e)}",
                "correctness_score": 0,
                "completeness_score": 0,
                "clarity_score": 0,
                "critical_thinking_score": 0,
                "total_score": 0,
                "marks_awarded": 0.0,
                "positive_feedback": "Unable to evaluate.",
                "areas_for_improvement": "Unable to evaluate.",
                "suggested_improvements": "Unable to evaluate."
            }
    
    def batch_evaluate(self, evaluations: List[Dict]) -> List[Dict]:
        """
        Batch evaluate multiple student answers.
        
        :param evaluations: List of dictionaries, each containing 'question', 'correct_answer', 'student_answer', and optional 'max_marks'
        :return: List of evaluation results
        """
        results = []
        for item in evaluations:
            try:
                max_marks = item.get('max_marks', 10.0)
                result = self.evaluate_answer(
                    item['question'], 
                    item['correct_answer'], 
                    item['student_answer'],
                    max_marks
                )
                results.append({
                    "question": item['question'],
                    "student_id": item.get('student_id', 'unknown'),
                    "max_marks": max_marks,
                    "evaluation": result
                })
            except Exception as e:
                logging.error(f"Error evaluating item: {e}")
                results.append({
                    "question": item['question'],
                    "student_id": item.get('student_id', 'unknown'),
                    "max_marks": item.get('max_marks', 10.0),
                    "evaluation": {"error": str(e)}
                })
        
        return results
    
    def generate_corrected_answer(self, question: str, student_answer: str) -> str:
        """
        Generate a corrected version of the student's answer.
        
        :param question: The question being answered
        :param student_answer: The student's submitted answer
        :return: A corrected version of the answer
        """
        correction_template = """
        Question: {question}
        
        Student Answer: {student_answer}
        
        Please provide a corrected and improved version of this answer that addresses any errors, 
        misconceptions, or omissions while maintaining the student's original approach where valid.
        The corrected answer should be comprehensive, accurate, and well-structured.
        """
        
        correction_prompt = PromptTemplate(
            input_variables=["question", "student_answer"],
            template=correction_template,
        )
        
        correction_chain = LLMChain(
            llm=self.llmss.llm,
            prompt=correction_prompt,
            verbose=False
        )
        
        try:
            result = correction_chain.invoke({
                "question": question,
                "student_answer": student_answer
            })
            
            return result['text']
        except Exception as e:
            logging.error(f"Error generating corrected answer: {e}")
            return f"Error generating correction: {str(e)}"


class AnswerEvaluationSystem:
    def __init__(self, llm: Optional[OllamaLLM] = None):
        """
        Initialize the complete Answer Evaluation System.
        
        :param llm: Optional pre-configured LLM
        """
        self.llmss = LLMSS(llm)
        self.evaluator = StudentAnswerEvaluator(self.llmss)
        logging.info("AnswerEvaluationSystem initialized successfully")
    
    def evaluate_submission(self, 
                           question: str, 
                           correct_answer: str, 
                           student_answer: str, 
                           max_marks: float = 10.0,
                           generate_correction: bool = False) -> Dict:
        """
        Evaluate a student submission and optionally generate a correction.
        
        :param question: The question being answered
        :param correct_answer: The correct or model answer
        :param student_answer: The student's submitted answer
        :param max_marks: Maximum marks for this question
        :param generate_correction: Whether to generate a corrected answer
        :return: Dictionary with evaluation and optional correction
        """
        evaluation = self.evaluator.evaluate_answer(question, correct_answer, student_answer, max_marks)
        
        result = {
            "evaluation": evaluation
        }
        
        if generate_correction:
            correction = self.evaluator.generate_corrected_answer(question, student_answer)
            result["corrected_answer"] = correction
        
        return result
    
    def process_submissions(self, 
                           submissions: List[Dict], 
                           generate_corrections: bool = False) -> Dict[str, List[Dict]]:
        """
        Process multiple student submissions.
        
        :param submissions: List of submission dictionaries
        :param generate_corrections: Whether to generate corrections
        :return: Dictionary with processed results
        """
        results = []
        
        for submission in submissions:
            try:
                max_marks = submission.get('max_marks', 10.0)
                result = self.evaluate_submission(
                    submission['question'],
                    submission['correct_answer'],
                    submission['student_answer'],
                    max_marks,
                    generate_corrections
                )
                
                results.append({
                    "student_id": submission.get('student_id', 'unknown'),
                    "question_id": submission.get('question_id', 'unknown'),
                    "max_marks": max_marks,
                    "result": result
                })
            except Exception as e:
                logging.error(f"Error processing submission: {e}")
                results.append({
                    "student_id": submission.get('student_id', 'unknown'),
                    "question_id": submission.get('question_id', 'unknown'),
                    "max_marks": submission.get('max_marks', 10.0),
                    "error": str(e)
                })
        
        return {
            "processed_count": len(results),
            "results": results
        }
    
    def generate_grade_report(self, student_results: List[Dict]) -> Dict:
        """
        Generate a comprehensive grade report from multiple evaluations for a student.
        
        :param student_results: List of evaluation results for a student
        :return: Dictionary containing grade report
        """
        if not student_results:
            return {
                "error": "No results provided for grade report generation"
            }
        
        try:
            # Extract student ID from first result
            student_id = student_results[0].get("student_id", "unknown")
            
            total_marks_earned = 0
            total_marks_possible = 0
            evaluations = []
            
            # Process each result
            for result in student_results:
                if "error" in result:
                    evaluations.append({
                        "question_id": result.get("question_id", "unknown"),
                        "error": result["error"]
                    })
                    continue
                
                max_marks = result.get("max_marks", 10.0)
                evaluation = result.get("result", {}).get("evaluation", {})
                
                marks_awarded = evaluation.get("marks_awarded", 0)
                total_marks_earned += marks_awarded
                total_marks_possible += max_marks
                
                evaluations.append({
                    "question_id": result.get("question_id", "unknown"),
                    "max_marks": max_marks,
                    "marks_awarded": marks_awarded,
                    "letter_grade": evaluation.get("letter_grade", "N/A"),
                    "feedback": {
                        "positive": evaluation.get("positive_feedback", ""),
                        "improvement": evaluation.get("areas_for_improvement", "")
                    }
                })
            
            # Calculate overall statistics
            overall_percentage = (total_marks_earned / total_marks_possible * 100) if total_marks_possible > 0 else 0
            overall_letter_grade = GradingSystem.determine_letter_grade(overall_percentage)
            passing = GradingSystem.is_passing(overall_percentage)
            
            return {
                "student_id": student_id,
                "overall_results": {
                    "total_marks_earned": round(total_marks_earned, 2),
                    "total_marks_possible": total_marks_possible,
                    "percentage": round(overall_percentage, 2),
                    "letter_grade": overall_letter_grade,
                    "passing": passing
                },
                "question_evaluations": evaluations
            }
        except Exception as e:
            logging.error(f"Error generating grade report: {e}")
            return {
                "student_id": student_id if 'student_id' in locals() else "unknown",
                "error": f"Failed to generate grade report: {str(e)}"
            }


# Example usage
if __name__ == "__main__":
    # Initialize the system
    evaluation_system = AnswerEvaluationSystem()
    
    # Example data
    question = "Explain the concept of photosynthesis and its importance for life on Earth."
    correct_answer = """
    Photosynthesis is the process by which green plants, algae, and some bacteria convert light energy, 
    usually from the sun, into chemical energy in the form of glucose or other sugars. The process primarily 
    takes place in the chloroplasts of plant cells, specifically using the green pigment chlorophyll.
    
    The basic equation for photosynthesis is: 
    6CO₂ + 6H₂O + light energy → C₆H₁₂O₆ (glucose) + 6O₂
    
    Photosynthesis is crucial for life on Earth for several reasons:
    1. It produces oxygen, which most organisms need for respiration
    2. It converts inorganic carbon (CO₂) to organic compounds, forming the base of food chains
    3. It helps regulate atmospheric CO₂ levels, playing a role in climate regulation
    4. It provides the energy that drives virtually all ecosystems
    """
    
    student_answer = """
    Photosynthesis is how plants make food using sunlight. They take in carbon dioxide and water and 
    produce oxygen and glucose. This is important because it gives us oxygen to breathe and food to eat.
    Plants have chlorophyll which makes them green and helps them capture sunlight.
    """
    
    # Evaluate a single submission with marks
    result = evaluation_system.evaluate_submission(
        question, 
        correct_answer, 
        student_answer,
        max_marks=20.0,  # This question is worth 20 marks
        generate_correction=True
    )
    
    print(json.dumps(result, indent=2))
    
    # Process multiple submissions with different mark values
    submissions = [
        {
            "student_id": "S12345",
            "question_id": "Q1",
            "question": question,
            "correct_answer": correct_answer,
            "student_answer": student_answer,
            "max_marks": 20.0
        },
        {
            "student_id": "S12345",
            "question_id": "Q2",
            "question": "Describe the process of cellular respiration.",
            "correct_answer": "Cellular respiration is the process by which cells convert glucose into energy in the form of ATP...",
            "student_answer": "Cellular respiration is how cells make energy from glucose. They use oxygen and produce CO2.",
            "max_marks": 15.0
        },
        {
            "student_id": "S67890",
            "question_id": "Q1",
            "question": question,
            "correct_answer": correct_answer,
            "student_answer": "Photosynthesis is the process of plants converting sunlight to energy.",
            "max_marks": 20.0
        }
    ]
    
    batch_results = evaluation_system.process_submissions(submissions, generate_corrections=True)
    print(f"Processed {batch_results['processed_count']} submissions")
    
    # Generate a grade report for student S12345
    student_results = [result for result in batch_results["results"] if result.get("student_id") == "S12345"]
    grade_report = evaluation_system.generate_grade_report(student_results)
    print(json.dumps(grade_report, indent=2))

INFO:root:LLMSS initialized successfully
INFO:root:StudentAnswerEvaluator initialized successfully
INFO:root:AnswerEvaluationSystem initialized successfully
INFO:root:Evaluating answer for question: Explain the concept of photosynthesis and its impo...
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
INFO:root:Evaluation complete. Total score: 23, Grade: F
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
INFO:root:Evaluating answer for question: Explain the concept of photosynthesis and its impo...


{
  "evaluation": {
    "correctness_score": 6,
    "completeness_score": 4,
    "clarity_score": 8,
    "critical_thinking_score": 5,
    "total_score": 23,
    "marks_awarded": 11.5,
    "positive_feedback": "The student demonstrated good understanding of the basic concept of photosynthesis, including its role in producing oxygen and glucose.",
    "areas_for_improvement": "The student could improve by providing more detail about the process, explaining the importance of chlorophyll, and discussing other critical aspects such as the role of CO\u2082 and O\u2082 in the equation.",
    "suggested_improvements": "A more complete answer would include a clear explanation of the photosynthesis equation and its significance in regulating atmospheric CO\u2082 levels.",
    "percentage": 57.5,
    "letter_grade": "F",
    "passing": false
  },
  "corrected_answer": "Here's a revised version of the student's answer:\n\nPhotosynthesis is the process by which plants, algae, and some bacteria con

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
INFO:root:Evaluation complete. Total score: 20, Grade: F
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
INFO:root:Evaluating answer for question: Describe the process of cellular respiration....
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
INFO:root:Evaluation complete. Total score: 22, Grade: F
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
INFO:root:Evaluating answer for question: Explain the concept of photosynthesis and its impo...
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
INFO:root:Evaluation complete. Total score: 28, Grade: C
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Processed 3 submissions
{
  "student_id": "S12345",
  "overall_results": {
    "total_marks_earned": 21.0,
    "total_marks_possible": 35.0,
    "percentage": 60.0,
    "letter_grade": "D",
    "passing": true
  },
  "question_evaluations": [
    {
      "question_id": "Q1",
      "max_marks": 20.0,
      "marks_awarded": 10.0,
      "letter_grade": "F",
      "feedback": {
        "positive": "The student demonstrated some good understanding of the basic process and its importance for human life.",
        "improvement": "The student could improve by providing more detail about the equation, explaining the role of chlorophyll in photosynthesis, and discussing other important reasons why photosynthesis is crucial for life on Earth."
      }
    },
    {
      "question_id": "Q2",
      "max_marks": 15.0,
      "marks_awarded": 11.0,
      "letter_grade": "F",
      "feedback": {
        "positive": "The student demonstrated a good understanding of the basic process of cellular respirat

In [12]:
from typing import Optional, Dict, List, Union, Tuple
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from AIAnswerEvaluationSystem.logger import logging
import json
import re
from neo4j import GraphDatabase

class Neo4jManager:
    """Manages connections and queries to Neo4j database"""
    
    def __init__(self, uri: str, username: str, password: str):
        """
        Initialize the Neo4j connection manager.
        
        :param uri: Neo4j database URI
        :param username: Neo4j username
        :param password: Neo4j password
        """
        try:
            self.driver = GraphDatabase.driver(uri, auth=(username, password))
            logging.info("Connected to Neo4j database successfully")
        except Exception as e:
            logging.error(f"Failed to connect to Neo4j: {e}")
            raise
    
    def close(self):
        """Close the Neo4j connection"""
        if hasattr(self, 'driver'):
            self.driver.close()
    
    def fetch_student_answers_with_correct_answers(self, student_roll_no: str, subject_name: str) -> List[Dict]:
        """
        Fetch student answers with corresponding correct answers from Neo4j.
        
        :param student_roll_no: Student's roll number
        :param subject_name: Subject name
        :return: List of dictionaries containing questions, student answers, and correct answers
        """
        query = """
        MATCH (s:Student {roll_no: $roll_no})
        MATCH (s)-[:SUBMITTED]->(e:ExamSubmission)-[:FOR_SUBJECT]->(sub:Subject {name: $subject_name})
        MATCH (e)-[:HAS_ANSWER]->(sa:Answer)-[:FOR_QUESTION]->(q:Question)
        WITH s, q, sa
        MATCH (sub:Subject)-[:HAS_QUESTION]->(q)-[:HAS_ANSWER]->(ca:Answer)
        WHERE sub.name = $subject_name
        RETURN 
            s.roll_no as student_roll,
            s.name as student_name,
            q.id as question_id,
            q.text as question_text,
            sa.text as student_answer,
            ca.text as correct_answer
        ORDER BY q.id
        """
        
        try:
            with self.driver.session() as session:
                result = session.run(query, roll_no=student_roll_no, subject_name=subject_name)
                records = [record.data() for record in result]
                
                if not records:
                    logging.warning(f"No answers found for student {student_roll_no} in subject {subject_name}")
                print(records)
                return records
        except Exception as e:
            logging.error(f"Error fetching answers from Neo4j: {e}")
            raise

class LLMSS:
    def __init__(self, llm: Optional[OllamaLLM] = None):
        """
        Initialize the LLMSS with an optional LLM.
        
        :param llm: Optional pre-configured OllamaLLM instance
        """
        try:
            self.llm = llm if llm is not None else OllamaLLM(model="llama3.2:latest")
            logging.info("LLMSS initialized successfully")
        except Exception as e:
            logging.error(f"Failed to initialize LLMSS: {e}")
            raise


class GradingSystem:
    """Handles converting evaluation scores to letter grades and determining pass/fail status."""
    
    # Grade scale mapping percentage ranges to letter grades
    GRADE_SCALE = {
        (90, 100): 'A',
        (80, 89.99): 'B',
        (70, 79.99): 'C',
        (60, 69.99): 'D',
        (0, 59.99): 'F'
    }
    
    # Minimum passing grade threshold (as a percentage)
    PASSING_THRESHOLD = 60.0
    
    @classmethod
    def calculate_percentage(cls, score: int, max_score: int) -> float:
        """
        Calculate percentage score.
        
        :param score: Achieved score
        :param max_score: Maximum possible score
        :return: Percentage as float
        """
        if max_score <= 0:
            logging.error("Invalid max_score (must be > 0)")
            return 0.0
        
        return (score / max_score) * 100
    
    @classmethod
    def determine_letter_grade(cls, percentage: float) -> str:
        """
        Convert percentage to letter grade.
        
        :param percentage: Score as percentage
        :return: Letter grade (A-F)
        """
        for score_range, grade in cls.GRADE_SCALE.items():
            if score_range[0] <= percentage <= score_range[1]:
                return grade
        
        logging.error(f"Unable to determine grade for percentage: {percentage}")
        return 'N/A'
    
    @classmethod
    def is_passing(cls, percentage: float) -> bool:
        """
        Determine if the score is passing.
        
        :param percentage: Score as percentage
        :return: True if passing, False otherwise
        """
        return percentage >= cls.PASSING_THRESHOLD
    
    @classmethod
    def grade_evaluation(cls, evaluation: Dict, max_score: int = 40) -> Dict:
        """
        Add grading information to an evaluation result.
        
        :param evaluation: Evaluation dictionary with scores
        :param max_score: Maximum possible score (default: 40 based on our scoring criteria)
        :return: Evaluation with added grading information
        """
        try:
            total_score = evaluation.get('total_score', 0)
            percentage = cls.calculate_percentage(total_score, max_score)
            letter_grade = cls.determine_letter_grade(percentage)
            passing = cls.is_passing(percentage)
            
            # Add grading information to the evaluation
            evaluation.update({
                'percentage': round(percentage, 2),
                'letter_grade': letter_grade,
                'passing': passing
            })
            
            return evaluation
        except Exception as e:
            logging.error(f"Error in grading evaluation: {e}")
            # Return original evaluation with error information
            evaluation.update({
                'grading_error': str(e)
            })
            return evaluation


class StudentAnswerEvaluator:
    def __init__(self, llmss: LLMSS):
        """
        Initialize the StudentAnswerEvaluator with an LLMSS instance.
        
        :param llmss: An initialized LLMSS instance
        """
        self.llmss = llmss
        self._setup_evaluation_chain()
        logging.info("StudentAnswerEvaluator initialized successfully")
    
    def _setup_evaluation_chain(self):
        """Set up the evaluation chain with appropriate prompt templates."""
        evaluation_template = """
        You are an expert educator tasked with evaluating student answers.
        
        Question: {question}
        
        Correct Answer: {correct_answer}
        
        Student Answer: {student_answer}
        
        Question Mark Value: {max_marks}
        
        Evaluate the student's answer based on the following criteria:
        1. Correctness (0-10): How factually correct is the answer?
        2. Completeness (0-10): Does the answer cover all required elements?
        3. Clarity (0-10): How clear and well-articulated is the answer?
        4. Critical Thinking (0-10): Does the answer demonstrate understanding and critical thinking?
        
        Provide a total score out of 40, which will be converted to a score out of the maximum marks ({max_marks}).
        
        Also provide:
        - Specific feedback on what was good
        - Specific feedback on what could be improved
        - Suggested correct answer or improvements
        
        Your response MUST be in valid JSON format like this:
        {{
            "correctness_score": 8,
            "completeness_score": 7,
            "clarity_score": 9,
            "critical_thinking_score": 6,
            "total_score": 30,
            "marks_awarded": 15.0,
            "positive_feedback": "The student demonstrated good understanding of...",
            "areas_for_improvement": "The student could improve by...",
            "suggested_improvements": "A more complete answer would include..."
        }}
        
        Do not include any text outside the JSON object. The JSON must be properly formatted with no trailing commas.
        """
        
        self.evaluation_prompt = PromptTemplate(
            input_variables=["question", "correct_answer", "student_answer", "max_marks"],
            template=evaluation_template,
        )
        
        self.evaluation_chain = LLMChain(
            llm=self.llmss.llm,
            prompt=self.evaluation_prompt,
            verbose=False
        )
    
    def _extract_json(self, text: str) -> Dict:
        """
        Extract a valid JSON object from text, with improved error handling.
        
        :param text: Text potentially containing JSON
        :return: Extracted JSON as dictionary or error dict
        """
        # First try direct parsing
        text = text.strip()
        try:
            return json.loads(text)
        except json.JSONDecodeError:
            pass  # Continue to other extraction methods
        
        # Look for JSON with regex - find content between curly braces
        try:
            json_match = re.search(r'({[\s\S]*?})(?:\s*$|\n)', text)
            if json_match:
                json_str = json_match.group(1)
                return json.loads(json_str)
        except (json.JSONDecodeError, AttributeError):
            pass
        
        # Try to fix common JSON formatting issues
        try:
            # Replace single quotes with double quotes
            fixed_text = text.replace("'", '"')
            return json.loads(fixed_text)
        except json.JSONDecodeError:
            pass
        
        # If all extraction methods fail, create a default evaluation
        logging.error(f"All JSON extraction methods failed. Raw text: {text[:100]}...")
        return {
            "error": "Failed to parse result as JSON",
            "raw_output": text
        }
    
    def evaluate_answer(self, question: str, correct_answer: str, student_answer: str, max_marks: float = 10.0) -> Dict:
        """
        Evaluate a student's answer against the correct answer for a given question.
        
        :param question: The question being answered
        :param correct_answer: The correct or model answer
        :param student_answer: The student's submitted answer
        :param max_marks: Maximum marks for this question
        :return: Dictionary containing evaluation scores and feedback
        """
        try:
            logging.info(f"Evaluating answer for question: {question[:50]}...")
            
            # Create a default evaluation with required fields
            default_evaluation = {
                "correctness_score": 0,
                "completeness_score": 0,
                "clarity_score": 0,
                "critical_thinking_score": 0,
                "total_score": 0,
                "marks_awarded": 0.0,
                "positive_feedback": "",
                "areas_for_improvement": "",
                "suggested_improvements": ""
            }
            
            result = self.evaluation_chain.invoke({
                "question": question,
                "correct_answer": correct_answer,
                "student_answer": student_answer,
                "max_marks": max_marks
            })
            
            # Extract JSON from the result with improved handling
            evaluation = self._extract_json(result['text'])
            
            # If there was an error in extraction, return the error info
            if "error" in evaluation:
                return evaluation
            
            # Update default evaluation with extracted values (ensures all keys exist)
            default_evaluation.update(evaluation)
            
            # Calculate marks awarded if not already in the result
            if 'marks_awarded' not in evaluation:
                default_evaluation['marks_awarded'] = (default_evaluation['total_score'] / 40) * max_marks
            
            # Apply grading to the evaluation
            graded_evaluation = GradingSystem.grade_evaluation(default_evaluation)
            
            logging.info(f"Evaluation complete. Total score: {graded_evaluation.get('total_score', 'N/A')}, Grade: {graded_evaluation.get('letter_grade', 'N/A')}")
            return graded_evaluation
        except Exception as e:
            logging.error(f"Error during answer evaluation: {e}")
            return {
                "error": f"Evaluation failed: {str(e)}",
                "correctness_score": 0,
                "completeness_score": 0,
                "clarity_score": 0,
                "critical_thinking_score": 0,
                "total_score": 0,
                "marks_awarded": 0.0,
                "positive_feedback": "Unable to evaluate.",
                "areas_for_improvement": "Unable to evaluate.",
                "suggested_improvements": "Unable to evaluate."
            }
    
    def batch_evaluate(self, evaluations: List[Dict]) -> List[Dict]:
        """
        Batch evaluate multiple student answers.
        
        :param evaluations: List of dictionaries, each containing 'question', 'correct_answer', 'student_answer', and optional 'max_marks'
        :return: List of evaluation results
        """
        results = []
        for item in evaluations:
            try:
                max_marks = item.get('max_marks', 10.0)
                result = self.evaluate_answer(
                    item['question'], 
                    item['correct_answer'], 
                    item['student_answer'],
                    max_marks
                )
                results.append({
                    "question": item['question'],
                    "student_id": item.get('student_id', 'unknown'),
                    "max_marks": max_marks,
                    "evaluation": result
                })
            except Exception as e:
                logging.error(f"Error evaluating item: {e}")
                results.append({
                    "question": item['question'],
                    "student_id": item.get('student_id', 'unknown'),
                    "max_marks": item.get('max_marks', 10.0),
                    "evaluation": {"error": str(e)}
                })
        
        return results
    
    def generate_corrected_answer(self, question: str, student_answer: str) -> str:
        """
        Generate a corrected version of the student's answer.
        
        :param question: The question being answered
        :param student_answer: The student's submitted answer
        :return: A corrected version of the answer
        """
        correction_template = """
        Question: {question}
        
        Student Answer: {student_answer}
        
        Please provide a corrected and improved version of this answer that addresses any errors, 
        misconceptions, or omissions while maintaining the student's original approach where valid.
        The corrected answer should be comprehensive, accurate, and well-structured.
        """
        
        correction_prompt = PromptTemplate(
            input_variables=["question", "student_answer"],
            template=correction_template,
        )
        
        correction_chain = LLMChain(
            llm=self.llmss.llm,
            prompt=correction_prompt,
            verbose=False
        )
        
        try:
            result = correction_chain.invoke({
                "question": question,
                "student_answer": student_answer
            })
            
            return result['text']
        except Exception as e:
            logging.error(f"Error generating corrected answer: {e}")
            return f"Error generating correction: {str(e)}"


class Neo4jAnswerEvaluationSystem:
    def __init__(self, neo4j_uri: str, neo4j_username: str, neo4j_password: str, llm: Optional[OllamaLLM] = None):
        """
        Initialize the Neo4j-based Answer Evaluation System.
        
        :param neo4j_uri: URI for Neo4j database
        :param neo4j_username: Neo4j username
        :param neo4j_password: Neo4j password
        :param llm: Optional pre-configured LLM
        """
        self.neo4j = Neo4jManager(neo4j_uri, neo4j_username, neo4j_password)
        self.llmss = LLMSS(llm)
        self.evaluator = StudentAnswerEvaluator(self.llmss)
        logging.info("Neo4jAnswerEvaluationSystem initialized successfully")
    
    def close(self):
        """Close Neo4j connection when system is no longer needed"""
        self.neo4j.close()
    
    def evaluate_student_subject(self, 
                                student_roll_no: str, 
                                subject_name: str, 
                                max_marks_per_question: float = 10.0,
                                generate_corrections: bool = False) -> Dict:
        """
        Evaluate all answers from a student for a specific subject.
        
        :param student_roll_no: Student's roll number
        :param subject_name: Subject name
        :param max_marks_per_question: Maximum marks per question (can be overridden)
        :param generate_corrections: Whether to generate corrections for answers
        :return: Dictionary with complete evaluation results
        """
        try:
            # Fetch student answers with correct answers from Neo4j
            records = self.neo4j.fetch_student_answers_with_correct_answers(student_roll_no, subject_name)
            
            if not records:
                return {
                    "status": "warning",
                    "message": f"No answers found for student {student_roll_no} in subject {subject_name}"
                }
            
            # Process each question and answer
            results = []
            for record in records:
                question_text = record.get("question_text", "")
                student_answer = record.get("student_answer", "")
                correct_answer = record.get("correct_answer", "")
                question_id = record.get("question_id", "unknown")
                
                # Evaluate the answer
                evaluation = self.evaluator.evaluate_answer(
                    question_text,
                    correct_answer,
                    student_answer,
                    max_marks_per_question
                )
                
                result = {
                    "question_id": question_id,
                    "evaluation": evaluation
                }
                
                # Generate correction if requested
                if generate_corrections:
                    correction = self.evaluator.generate_corrected_answer(question_text, student_answer)
                    result["corrected_answer"] = correction
                
                results.append(result)
            
            # Generate the complete grade report
            student_name = records[0].get("student_name", "Unknown Student")
            grade_report = self.generate_grade_report(results, student_roll_no, student_name, subject_name)
            
            return {
                "status": "success",
                "student_roll": student_roll_no,
                "student_name": student_name,
                "subject": subject_name,
                "question_count": len(results),
                "grade_report": grade_report,
                "question_evaluations": results
            }
        except Exception as e:
            logging.error(f"Error evaluating student subject: {e}")
            return {
                "status": "error",
                "message": f"Failed to evaluate answers: {str(e)}",
                "student_roll": student_roll_no,
                "subject": subject_name
            }
    
    def generate_grade_report(self, 
                              evaluations: List[Dict], 
                              student_roll: str, 
                              student_name: str, 
                              subject_name: str) -> Dict:
        """
        Generate a comprehensive grade report from evaluations.
        
        :param evaluations: List of evaluation results
        :param student_roll: Student's roll number
        :param student_name: Student's name
        :param subject_name: Subject name
        :return: Dictionary containing grade report
        """
        if not evaluations:
            return {
                "error": "No evaluations provided for grade report generation"
            }
        
        try:
            total_marks_earned = 0
            total_marks_possible = 0
            question_summaries = []
            
            # Process each evaluation
            for result in evaluations:
                evaluation = result.get("evaluation", {})
                if "error" in evaluation:
                    question_summaries.append({
                        "question_id": result.get("question_id", "unknown"),
                        "error": evaluation["error"]
                    })
                    continue
                
                marks_awarded = evaluation.get("marks_awarded", 0)
                max_marks = evaluation.get("max_marks", 10.0)
                
                total_marks_earned += marks_awarded
                total_marks_possible += max_marks
                
                question_summaries.append({
                    "question_id": result.get("question_id", "unknown"),
                    "marks_awarded": marks_awarded,
                    "max_marks": max_marks,
                    "letter_grade": evaluation.get("letter_grade", "N/A"),
                    "feedback": {
                        "positive": evaluation.get("positive_feedback", ""),
                        "improvement": evaluation.get("areas_for_improvement", "")
                    }
                })
            
            # Calculate overall statistics
            overall_percentage = (total_marks_earned / total_marks_possible * 100) if total_marks_possible > 0 else 0
            overall_letter_grade = GradingSystem.determine_letter_grade(overall_percentage)
            passing = GradingSystem.is_passing(overall_percentage)
            
            return {
                "student_roll": student_roll,
                "student_name": student_name,
                "subject": subject_name,
                "overall_results": {
                    "total_marks_earned": round(total_marks_earned, 2),
                    "total_marks_possible": total_marks_possible,
                    "percentage": round(overall_percentage, 2),
                    "letter_grade": overall_letter_grade,
                    "passing": passing
                },
                "question_summaries": question_summaries,
                "generated_at": "datetime.now().isoformat()"  # This would be actual datetime in real code
            }
        except Exception as e:
            logging.error(f"Error generating grade report: {e}")
            return {
                "error": f"Failed to generate grade report: {str(e)}",
                "student_roll": student_roll,
                "student_name": student_name,
                "subject": subject_name
            }
    
    def batch_evaluate_multiple_students(self, 
                                        student_roll_numbers: List[str], 
                                        subject_name: str,
                                        max_marks_per_question: float = 10.0,
                                        generate_corrections: bool = False) -> Dict:
        """
        Batch evaluate multiple students for a specific subject.
        
        :param student_roll_numbers: List of student roll numbers
        :param subject_name: Subject name
        :param max_marks_per_question: Maximum marks per question
        :param generate_corrections: Whether to generate corrections
        :return: Dictionary with batch results
        """
        results = []
        
        for roll_no in student_roll_numbers:
            try:
                result = self.evaluate_student_subject(
                    roll_no,
                    subject_name,
                    max_marks_per_question,
                    generate_corrections
                )
                results.append(result)
            except Exception as e:
                logging.error(f"Error evaluating student {roll_no}: {e}")
                results.append({
                    "status": "error",
                    "message": f"Failed to evaluate student {roll_no}: {str(e)}",
                    "student_roll": roll_no,
                    "subject": subject_name
                })
        
        return {
            "status": "complete",
            "subject": subject_name,
            "student_count": len(results),
            "successful_evaluations": sum(1 for r in results if r.get("status") == "success"),
            "results": results
        }


# Example usage
if __name__ == "__main__":
    # Database connection credentials
    NEO4J_URI = "bolt://localhost:7687"  # Replace with actual Neo4j URI
    NEO4J_USERNAME = "neo4j"  # Replace with actual Neo4j username
    NEO4J_PASSWORD = "akashgola"  # Replace with actual Neo4j password
    
    # Initialize the system
    evaluation_system = Neo4jAnswerEvaluationSystem(
        neo4j_uri=NEO4J_URI,
        neo4j_username=NEO4J_USERNAME,
        neo4j_password=NEO4J_PASSWORD
    )
    
    try:
        # Evaluate a single student's answers for Artificial Intelligence
        result = evaluation_system.evaluate_student_subject(
            student_roll_no="2115500009",
            subject_name="Artificial Intelligence",
            max_marks_per_question=10.0,
            generate_corrections=True
        )
        
        print(json.dumps(result, indent=2))
        
        # Batch evaluate multiple students
        student_rolls = ["2115500007", "2115500009"]  # Add actual roll numbers
        batch_results = evaluation_system.batch_evaluate_multiple_students(
            student_roll_numbers=student_rolls,
            subject_name="Artificial Intelligence",
            max_marks_per_question=10.0,
            generate_corrections=False
        )
        
        print(f"Evaluated {batch_results['student_count']} students")
        print(f"Successful evaluations: {batch_results['successful_evaluations']}")
    finally:
        # Always close the Neo4j connection when done
        evaluation_system.close()

[{'student_roll': '2115500009', 'student_name': 'Akash', 'question_id': 'AI001_Q1', 'question_text': 'Define Artificial Intelligence', 'student_answer': 'dkujcdkjcbdcnidsvduvd', 'correct_answer': 'AI simulates human intelligence in machines'}]
{
  "status": "success",
  "student_roll": "2115500009",
  "student_name": "Akash",
  "subject": "Artificial Intelligence",
  "question_count": 1,
  "grade_report": {
    "student_roll": "2115500009",
    "student_name": "Akash",
    "subject": "Artificial Intelligence",
    "overall_results": {
      "total_marks_earned": 0.0,
      "total_marks_possible": 10.0,
      "percentage": 0.0,
      "letter_grade": "F",
      "passing": false
    },
    "question_summaries": [
      {
        "question_id": "AI001_Q1",
        "marks_awarded": 0.0,
        "max_marks": 10.0,
        "letter_grade": "F",
        "feedback": {
          "positive": "",
          "improvement": ""
        }
      }
    ],
    "generated_at": "datetime.now().isoformat()"
 

In [15]:
from typing import Optional, Dict, List, Union, Tuple
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from AIAnswerEvaluationSystem.logger import logging
import json
import re
from neo4j import GraphDatabase
from datetime import datetime

# --------------------------- Database Manager ---------------------------
class Neo4jManager:
    """Manages connections and queries to Neo4j database"""
    
    def __init__(self, uri: str, username: str, password: str):
        self.driver = self._init_driver(uri, username, password)
    
    def _init_driver(self, uri: str, username: str, password: str):
        """Initialize Neo4j driver with error handling"""
        try:
            driver = GraphDatabase.driver(uri, auth=(username, password))
            logging.info("Connected to Neo4j database successfully")
            return driver
        except Exception as e:
            logging.error(f"Failed to connect to Neo4j: {e}")
            raise
    
    def close(self):
        """Close the Neo4j connection"""
        if hasattr(self, 'driver'):
            self.driver.close()
    
    def submit_student_answer(self, student_roll: str, subject: str, question_id: str, answer: str) -> bool:
        """Submit a student answer to the database"""
        query = """
        MERGE (s:Student {roll_no: $roll})
        MERGE (sub:Subject {name: $subject})
        MERGE (q:Question {id: $qid})
        CREATE (s)-[:SUBMITTED]->(:ExamSubmission {
            timestamp: datetime()
        })-[:HAS_ANSWER]->(:Answer {
            text: $answer
        })-[:FOR_QUESTION]->(q)
        """
        try:
            with self.driver.session() as session:
                session.run(query, {
                    'roll': student_roll,
                    'subject': subject,
                    'qid': question_id,
                    'answer': answer
                })
            return True
        except Exception as e:
            logging.error(f"Submission failed: {e}")
            return False
    
    def fetch_answers(self, student_roll: str, subject: str) -> List[Dict]:
        """Fetch student answers with correct answers"""
        query = """
        MATCH (s:Student {roll_no: $roll})-[:SUBMITTED]->()-[:HAS_ANSWER]->(a:Answer)-[:FOR_QUESTION]->(q:Question)
        MATCH (q)-[:HAS_ANSWER]->(ca:Answer)
        RETURN q.text as question, a.text as student_answer, ca.text as correct_answer
        """
        try:
            with self.driver.session() as session:
                result = session.run(query, {'roll': student_roll, 'subject': subject})
                return [dict(record) for record in result]
        except Exception as e:
            logging.error(f"Fetch failed: {e}")
            return []
    
    def store_evaluation(self, student_roll: str, subject: str, evaluation: Dict) -> bool:
        """Store evaluation results in the database"""
        query = """
        MATCH (s:Student {roll_no: $roll})-[:SUBMITTED]->(e:ExamSubmission)
        SET e.evaluated = true,
            e.total_score = $total,
            e.grade = $grade,
            e.passing = $passing,
            e.evaluation_date = datetime()
        """
        try:
            with self.driver.session() as session:
                session.run(query, {
                    'roll': student_roll,
                    'total': evaluation['total_score'],
                    'grade': evaluation['letter_grade'],
                    'passing': evaluation['passing']
                })
            return True
        except Exception as e:
            logging.error(f"Evaluation storage failed: {e}")
            return False

# --------------------------- AI Evaluation System ---------------------------
class GradingSystem:
    """Handles score-to-grade conversions and pass/fail determinations"""
    
    GRADE_SCALE = {
        (90, 100): 'A', (80, 89.99): 'B',
        (70, 79.99): 'C', (60, 69.99): 'D',
        (0, 59.99): 'F'
    }
    PASSING_THRESHOLD = 60.0

    @classmethod
    def calculate_percentage(cls, score: int, max_score: int) -> float:
        return (score / max_score) * 100 if max_score else 0.0

    @classmethod
    def determine_grade(cls, percentage: float) -> Tuple[str, bool]:
        for (low, high), grade in cls.GRADE_SCALE.items():
            if low <= percentage <= high:
                return grade, percentage >= cls.PASSING_THRESHOLD
        return 'N/A', False

class AnswerEvaluator:
    """Handles answer evaluation using LLM"""
    
    def __init__(self, model: str = "llama3.2:latest"):
        self.llm = OllamaLLM(model=model)
        self.eval_chain = self._create_evaluation_chain()
    
    def _create_evaluation_chain(self) -> LLMChain:
        template = """[INST]Evaluate student answer:
        Question: {question}
        Correct Answer: {correct_answer}
        Student Answer: {student_answer}
        
        Scoring Criteria (0-10 points each):
        - Correctness
        - Completeness  
        - Clarity
        - Critical Thinking
        
        Return JSON format:
        {{
            "scores": {{
                "correctness": 8,
                "completeness": 7,
                "clarity": 6,
                "critical_thinking": 5
            }},
            "total": 26,
            "feedback": {{
                "strengths": "Good understanding of...",
                "weaknesses": "Needs improvement in...",
                "suggestions": "Consider adding..."
            }}
        }}[/INST]"""
        
        return LLMChain(
            llm=self.llm,
            prompt=PromptTemplate.from_template(template),
            verbose=False
        )
    
    def evaluate(self, question: str, correct: str, student: str) -> Dict:
        try:
            response = self.eval_chain.invoke({
                'question': question,
                'correct_answer': correct,
                'student_answer': student
            })
            return self._parse_response(response['text'])
        except Exception as e:
            logging.error(f"Evaluation error: {e}")
            return {'error': str(e)}

    def _parse_response(self, text: str) -> Dict:
        try:
            return json.loads(text.split('```json')[-1].split('```')[0])
        except:
            logging.warning("Using fallback JSON parsing")
            return json.loads(re.search(r'\{.*\}', text, re.DOTALL).group())

# --------------------------- Main Application ---------------------------
class AutoGradingSystem:
    """Main application handling submission and evaluation workflow"""
    
    def __init__(self, neo4j_uri: str, neo4j_auth: Tuple[str, str]):
        self.db = Neo4jManager(*neo4j_auth, neo4j_uri)
        self.evaluator = AnswerEvaluator()
    
    def submit_and_evaluate(self, student_roll: str, subject: str, 
                          qa_pairs: List[Dict]) -> Dict:
        """Submit answers and trigger immediate evaluation"""
        # Store answers
        for qa in qa_pairs:
            self.db.submit_student_answer(
                student_roll, subject,
                qa['question_id'], qa['answer']
            )
        
        # Evaluate all answers
        answers = self.db.fetch_answers(student_roll, subject)
        total = 0
        max_score = len(answers) * 10  # Assuming 10 points per question
        
        evaluations = []
        for record in answers:
            evaluation = self.evaluator.evaluate(
                record['question'],
                record['correct_answer'],
                record['student_answer']
            )
            evaluations.append(evaluation)
            total += evaluation.get('total', 0)
        
        # Calculate final grade
        percentage = GradingSystem.calculate_percentage(total, max_score)
        grade, passing = GradingSystem.determine_grade(percentage)
        
        # Store final evaluation
        result = {
            'total_score': total,
            'percentage': percentage,
            'letter_grade': grade,
            'passing': passing,
            'evaluations': evaluations
        }
        self.db.store_evaluation(student_roll, subject, result)
        
        return result
    
    def check_results(self, student_roll: str, subject: str) -> Dict:
        """Retrieve stored evaluation results"""
        query = """
        MATCH (s:Student {roll_no: $roll})-[:SUBMITTED]->(e:ExamSubmission)
        RETURN e { .total_score, .grade, .passing, .evaluation_date } as result
        ORDER BY e.evaluation_date DESC LIMIT 1
        """
        try:
            with self.db.driver.session() as session:
                result = session.run(query, {'roll': student_roll, 'subject': subject})
                return [dict(record['result']) for record in result][0]
        except:
            return {'error': 'No results found'}

# --------------------------- Usage Example ---------------------------
if __name__ == "__main__":
    # Configuration
    NEO4J_URI = "neo4j://localhost:7687"
    NEO4J_AUTH = ("neo4j", "akashgola")
    
    # Initialize system
    grading_system = AutoGradingSystem(NEO4J_URI, NEO4J_AUTH)
    
    # Example submission
    submission = {
        'student_roll': "2115500009",
        'subject': "Artificial Intelligence",
        'qa_pairs': [
            {'question_id': "AI-101", 'answer': "ML is a subset of AI focusing on algorithms..."},
            {'question_id': "AI-102", 'answer': "Neural networks are computational models..."}
        ]
    }
    
    # Submit and evaluate
    results = grading_system.submit_and_evaluate(**submission)
    print(f"Immediate Evaluation Results: {json.dumps(results, indent=2)}")
    
    # Check stored results later
    stored_results = grading_system.check_results("2115500009", "Artificial Intelligence")
    print(f"Stored Evaluation: {json.dumps(stored_results, indent=2)}")
    
    # Cleanup
    grading_system.db.close()

ConfigurationError: URI scheme '' is not supported. Supported URI schemes are ['bolt', 'bolt+ssc', 'bolt+s', 'neo4j', 'neo4j+ssc', 'neo4j+s']. Examples: bolt://host[:port] or neo4j://host[:port][?routing_context]

In [18]:
from neo4j import GraphDatabase
import uuid

class ExamDatabase:
    def __init__(self, uri, username, password):
        self.driver = GraphDatabase.driver(uri, auth=(username, password))
        
    def close(self):
        self.driver.close()
        
    def create_or_get_subject(self, subject_name):
        with self.driver.session() as session:
            result = session.write_transaction(
                self._create_or_get_subject_node, subject_name)
            return result
            
    @staticmethod
    def _create_or_get_subject_node(tx, subject_name):
        # First check if subject exists
        query_check = (
            "MATCH (s:Subject {name: $subject_name}) "
            "RETURN s.id AS subject_id"
        )
        result = tx.run(query_check, subject_name=subject_name)
        record = result.single()
        
        # If subject exists, return its ID
        if record:
            return record["subject_id"]
        
        # Otherwise create a new subject
        query_create = (
            "CREATE (s:Subject {name: $subject_name, id: $id}) "
            "RETURN s.id AS subject_id"
        )
        result = tx.run(query_create, subject_name=subject_name, id=str(uuid.uuid4()))
        return result.single()["subject_id"]
        
    def create_question(self, subject_id, question_text, answer_text, context=None, tags=None):
        with self.driver.session() as session:
            result = session.write_transaction(
                self._create_question_node, subject_id, question_text, answer_text, context, tags)
            return result
            
    @staticmethod
    def _create_question_node(tx, subject_id, question_text, answer_text, context, tags):
        question_id = str(uuid.uuid4())
        
        # Create question node
        query = (
            "MATCH (s:Subject {id: $subject_id}) "
            "CREATE (q:Question {id: $question_id, text: $question_text}) "
            "CREATE (a:Answer {id: $answer_id, text: $answer_text}) "
            "CREATE (s)-[:CONTAINS]->(q) "
            "CREATE (q)-[:HAS_ANSWER]->(a) "
        )
        
        params = {
            "subject_id": subject_id,
            "question_id": question_id,
            "question_text": question_text,
            "answer_id": str(uuid.uuid4()),
            "answer_text": answer_text
        }
        
        # Add context if provided
        if context:
            query += (
                "CREATE (c:Context {id: $context_id, text: $context_text}) "
                "CREATE (q)-[:HAS_CONTEXT]->(c) "
            )
            params["context_id"] = str(uuid.uuid4())
            params["context_text"] = context
            
        # Add tags if provided
        if tags and len(tags) > 0:
            for i, tag in enumerate(tags):
                tag_param = f"tag_{i}"
                query += (
                    f"MERGE (t{i}:Tag {{name: ${tag_param}}}) "
                    f"CREATE (q)-[:TAGGED_WITH]->(t{i}) "
                )
                params[tag_param] = tag
        
        query += "RETURN q.id AS question_id"
        
        result = tx.run(query, **params)
        return result.single()["question_id"]
    
    def link_related_questions(self, question_id1, question_id2, relationship_type="RELATED_TO"):
        with self.driver.session() as session:
            session.write_transaction(
                self._create_question_relationship, question_id1, question_id2, relationship_type)
    
    @staticmethod
    def _create_question_relationship(tx, question_id1, question_id2, relationship_type):
        # Check if relationship already exists to avoid duplicates
        check_query = (
            f"MATCH (q1:Question {{id: $qid1}})-[r:{relationship_type}]->(q2:Question {{id: $qid2}}) "
            f"RETURN COUNT(r) AS relationship_count"
        )
        result = tx.run(check_query, qid1=question_id1, qid2=question_id2)
        if result.single()["relationship_count"] > 0:
            return  # Relationship already exists
        
        # Create relationship if it doesn't exist
        query = (
            f"MATCH (q1:Question {{id: $qid1}}), (q2:Question {{id: $qid2}}) "
            f"WHERE q1 <> q2 "
            f"CREATE (q1)-[:{relationship_type}]->(q2) "
        )
        tx.run(query, qid1=question_id1, qid2=question_id2)
    
    def get_exam_by_subject(self, subject_name):
        with self.driver.session() as session:
            result = session.read_transaction(self._get_questions_by_subject, subject_name)
            return result
            
    @staticmethod
    def _get_questions_by_subject(tx, subject_name):
        query = (
            "MATCH (s:Subject {name: $subject_name})-[:CONTAINS]->(q:Question) "
            "MATCH (q)-[:HAS_ANSWER]->(a) "
            "OPTIONAL MATCH (q)-[:HAS_CONTEXT]->(c) "
            "OPTIONAL MATCH (q)-[:TAGGED_WITH]->(t) "
            "RETURN q.id AS qid, q.text AS question, a.text AS answer, "
            "c.text AS context, collect(DISTINCT t.name) AS tags"
        )
        result = tx.run(query, subject_name=subject_name)
        return [record for record in result]
    
    def get_related_questions(self, question_id):
        with self.driver.session() as session:
            result = session.read_transaction(self._get_related_questions, question_id)
            return result
    
    @staticmethod
    def _get_related_questions(tx, question_id):
        query = (
            "MATCH (q:Question {id: $qid})-[r]->(related:Question) "
            "RETURN related.id AS related_id, related.text AS related_text, "
            "type(r) AS relationship_type"
        )
        result = tx.run(query, qid=question_id)
        return [record for record in result]
    
    def get_all_subjects(self):
        with self.driver.session() as session:
            result = session.read_transaction(self._get_all_subjects)
            return result
    
    @staticmethod
    def _get_all_subjects(tx):
        query = "MATCH (s:Subject) RETURN s.id AS id, s.name AS name"
        result = tx.run(query)
        return [{"id": record["id"], "name": record["name"]} for record in result]


class ExamCreator:
    def __init__(self, db_connection):
        self.db = db_connection
        
    def create_or_get_subject(self, subject_name):
        """Create a subject if it doesn't exist, or get existing subject id"""
        return self.db.create_or_get_subject(subject_name)
        
    def add_question(self, subject_id, question_text, answer_text, context=None, tags=None):
        return self.db.create_question(subject_id, question_text, answer_text, context, tags)
    
    def link_questions(self, question_id1, question_id2, relationship_type="RELATED_TO"):
        self.db.link_related_questions(question_id1, question_id2, relationship_type)
        
    def generate_exam(self, subject_name):
        questions = self.db.get_exam_by_subject(subject_name)
        exam_data = {
            "subject": subject_name,
            "questions": []
        }
        
        for q in questions:
            question_data = {
                "qid": q["qid"],
                "question": q["question"],
                "answer": q["answer"]
            }
            
            if q["context"]:
                question_data["context"] = q["context"]
                
            if q["tags"] and len(q["tags"]) > 0:
                question_data["tags"] = q["tags"]
                
            # Get related questions
            related = self.db.get_related_questions(q["qid"])
            if related and len(related) > 0:
                question_data["related"] = [{
                    "id": r["related_id"],
                    "text": r["related_text"],
                    "relationship": r["relationship_type"]
                } for r in related]
                
            exam_data["questions"].append(question_data)
            
        return exam_data
    
    def list_all_subjects(self):
        """Get all subjects from the database"""
        return self.db.get_all_subjects()


# Example usage
def main():
    # Connect to Neo4j database
    db = ExamDatabase("neo4j://localhost:7687", "neo4j", "akashgola")
    
    # Create exam creator
    exam_creator = ExamCreator(db)
    
    # Get or create subject (won't create duplicate subjects)
    math_subject_id = exam_creator.create_or_get_subject("Mathematics")
    print(f"Working with subject ID: {math_subject_id}")
    
    # List all subjects in the database
    subjects = exam_creator.list_all_subjects()
    print("Available subjects:")
    for subject in subjects:
        print(f"- {subject['name']} (ID: {subject['id']})")
    
    # Add questions to the subject with tags
    q1_id = exam_creator.add_question(
        math_subject_id,
        "What is the formula for the area of a circle?",
        "A = πr²",
        "r represents the radius of the circle",
        ["geometry", "formulas"]
    )
    
    q2_id = exam_creator.add_question(
        math_subject_id,
        "Solve for x in the equation 2x + 5 = 13",
        "x = 4",
        "Subtract 5 from both sides, then divide by 2",
        ["algebra", "equations"]
    )
    
    q3_id = exam_creator.add_question(
        math_subject_id,
        "What is the formula for the circumference of a circle?",
        "C = 2πr",
        "r represents the radius of the circle",
        ["geometry", "formulas"]
    )
    
    # Link related questions (with duplicate prevention)
    exam_creator.link_questions(q1_id, q3_id, "SAME_TOPIC")
    
    # Generate an exam
    math_exam = exam_creator.generate_exam("Mathematics")
    
    # Print the exam
    print(f"Subject: {math_exam['subject']}")
    for q in math_exam["questions"]:
        print(f"Question ID: {q['qid']}")
        print(f"Question: {q['question']}")
        print(f"Answer: {q['answer']}")
        if 'context' in q:
            print(f"Context: {q['context']}")
        if 'tags' in q:
            print(f"Tags: {', '.join(q['tags'])}")
        if 'related' in q:
            print("Related Questions:")
            for rel in q['related']:
                print(f"  - {rel['text']} (Relationship: {rel['relationship']})")
        print("---")
    
    # Close database connection
    db.close()


if __name__ == "__main__":
    main()

Working with subject ID: MATH001
Available subjects:
- Mathematics (ID: MATH001)
- Physics (ID: PHY001)
- Chemistry (ID: CHEM001)
- Artificial Intelligence (ID: AI001)
- Hindi (ID: HIN001)
Subject: Mathematics
Question ID: 9451b5b2-7d30-4701-a7da-d3e6ef836105
Question: What is the formula for the circumference of a circle?
Answer: C = 2πr
Context: r represents the radius of the circle
Tags: geometry, formulas
---
Question ID: e9291723-e794-42e8-864b-00cee9b7c296
Question: Solve for x in the equation 2x + 5 = 13
Answer: x = 4
Context: Subtract 5 from both sides, then divide by 2
Tags: equations, algebra
---
Question ID: 74e9ef38-2ac6-40a7-b906-c262f87666d5
Question: What is the formula for the area of a circle?
Answer: A = πr²
Context: r represents the radius of the circle
Tags: formulas, geometry
Related Questions:
  - What is the formula for the circumference of a circle? (Relationship: SAME_TOPIC)
---
Question ID: cc6e50be-86be-4a08-92ed-65e46f92a307
Question: What is the formula for

/tmp/ipykernel_6185/2430201336.py:13: DeprecationWarning: write_transaction has been renamed to execute_write
  result = session.write_transaction(
/tmp/ipykernel_6185/2430201336.py:149: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(self._get_all_subjects)
/tmp/ipykernel_6185/2430201336.py:41: DeprecationWarning: write_transaction has been renamed to execute_write
  result = session.write_transaction(
/tmp/ipykernel_6185/2430201336.py:92: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(
/tmp/ipykernel_6185/2430201336.py:116: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(self._get_questions_by_subject, subject_name)
/tmp/ipykernel_6185/2430201336.py:134: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(self._get_related_questions, question_id)


In [21]:
# from neo4j import GraphDatabase
# import uuid

# class ExamGraph:
#     def __init__(self, uri, user, password):
#         self.driver = GraphDatabase.driver(uri, auth=(user, password))

#     def close(self):
#         self.driver.close()

#     def add_question(self, subject_name, question_text, answer_text):
#         with self.driver.session() as session:
#             subject_id = session.write_transaction(self._create_or_get_subject_node, subject_name)
#             question_id, answer_id = session.write_transaction(self._create_question_node, subject_id, question_text, answer_text)
#             session.write_transaction(self._link_all_questions, subject_id, question_id)

#     @staticmethod
#     def _create_or_get_subject_node(tx, subject_name):
#         query = (
#             "MERGE (s:Subject {name: $subject_name}) "
#             "ON CREATE SET s.id = $id "
#             "RETURN s.id AS subject_id"
#         )
#         result = tx.run(query, subject_name=subject_name, id=str(uuid.uuid4()))
#         return result.single()["subject_id"]

#     @staticmethod
#     def _create_question_node(tx, subject_id, question_text, answer_text):
#         question_id = str(uuid.uuid4())
#         answer_id = str(uuid.uuid4())
        
#         query = (
#             "MATCH (s:Subject {id: $subject_id}) "
#             "CREATE (q:Question {id: $question_id, text: $question_text}) "
#             "CREATE (s)-[:CONTAINS]->(q) "
#             "CREATE (a:Answer {id: $answer_id, text: $answer_text}) "
#             "CREATE (q)-[:HAS_ANSWER]->(a) "
#             "RETURN q.id, a.id"
#         )
        
#         result = tx.run(query, subject_id=subject_id, question_id=question_id, question_text=question_text, answer_id=answer_id, answer_text=answer_text)
#         return result.single()["q.id"], result.single()["a.id"]

#     @staticmethod
#     def _link_all_questions(tx, subject_id, new_question_id):
#         query = (
#             "MATCH (s:Subject {id: $subject_id})-[:CONTAINS]->(q:Question), "
#             "(s)-[:CONTAINS]->(new_q:Question {id: $new_question_id}) "
#             "WHERE q.id <> new_q.id "
#             "MERGE (q)-[:RELATED_TO]->(new_q)"
#         )
#         tx.run(query, subject_id=subject_id, new_question_id=new_question_id)

#     def generate_exam(self, subject_name):
#         with self.driver.session() as session:
#             return session.read_transaction(self._get_exam_questions, subject_name)

#     @staticmethod
#     def _get_exam_questions(tx, subject_name):
#         query = (
#             "MATCH (s:Subject {name: $subject_name})-[:CONTAINS]->(q:Question) "
#             "OPTIONAL MATCH (q)-[:HAS_ANSWER]->(a:Answer) "
#             "RETURN q.id AS question_id, q.text AS question_text, a.text AS answer_text"
#         )
#         result = tx.run(query, subject_name=subject_name)
#         return [{"id": record["question_id"], "question": record["question_text"], "answer": record["answer_text"]} for record in result]
    



from neo4j import GraphDatabase
import uuid

class ExamDatabase:
    def __init__(self, uri, username, password):
        self.driver = GraphDatabase.driver(uri, auth=(username, password))
        
    def close(self):
        self.driver.close()
        
    def create_or_get_subject(self, subject_name):
        with self.driver.session() as session:
            return session.write_transaction(self._create_or_get_subject_node, subject_name)
            
    @staticmethod
    def _create_or_get_subject_node(tx, subject_name):
        query = (
            "MERGE (s:Subject {name: $subject_name}) "
            "ON CREATE SET s.id = $id "
            "RETURN s.id AS subject_id"
        )
        result = tx.run(query, subject_name=subject_name, id=str(uuid.uuid4()))
        return result.single()["subject_id"]
        
    def create_question(self, subject_id, question_text, answer_text, context=None, tags=None):
        with self.driver.session() as session:
            return session.write_transaction(
                self._create_question_node, subject_id, question_text, answer_text, context, tags)
            
    @staticmethod
    def _create_question_node(tx, subject_id, question_text, answer_text, context, tags):
        question_id = str(uuid.uuid4())
        answer_id = str(uuid.uuid4())
        query = (
            "MATCH (s:Subject {id: $subject_id}) "
            "CREATE (q:Question {id: $question_id, text: $question_text})-[:HAS_ANSWER]->(a:Answer {id: $answer_id, text: $answer_text}) "
            "CREATE (s)-[:CONTAINS]->(q) "
        )
        params = {
            "subject_id": subject_id,
            "question_id": question_id,
            "question_text": question_text,
            "answer_id": answer_id,
            "answer_text": answer_text
        }
        if context:
            context_id = str(uuid.uuid4())
            query += "CREATE (q)-[:HAS_CONTEXT]->(c:Context {id: $context_id, text: $context_text}) "
            params.update({"context_id": context_id, "context_text": context})
        if tags:
            for i, tag in enumerate(tags):
                tag_param = f"tag_{i}"
                query += f"MERGE (t{i}:Tag {{name: ${tag_param}}}) CREATE (q)-[:TAGGED_WITH]->(t{i}) "
                params[tag_param] = tag
        query += "RETURN q.id AS question_id"
        result = tx.run(query, **params)
        return result.single()["question_id"]
    
    def link_all_questions(self, subject_id):
        with self.driver.session() as session:
            session.write_transaction(self._link_questions_in_subject, subject_id)
    
    @staticmethod
    def _link_questions_in_subject(tx, subject_id):
        query = (
            "MATCH (s:Subject {id: $subject_id})-[:CONTAINS]->(q1:Question), "
            "(s)-[:CONTAINS]->(q2:Question) "
            "WHERE q1 <> q2 "
            "MERGE (q1)-[:RELATED_TO]->(q2)"
        )
        tx.run(query, subject_id=subject_id)
    
    def get_exam_by_subject(self, subject_name):
        with self.driver.session() as session:
            return session.read_transaction(self._get_questions_by_subject, subject_name)
            
    @staticmethod
    def _get_questions_by_subject(tx, subject_name):
        query = (
            "MATCH (s:Subject {name: $subject_name})-[:CONTAINS]->(q:Question) "
            "MATCH (q)-[:HAS_ANSWER]->(a) "
            "OPTIONAL MATCH (q)-[:HAS_CONTEXT]->(c) "
            "OPTIONAL MATCH (q)-[:TAGGED_WITH]->(t) "
            "RETURN q.id AS qid, q.text AS question, a.text AS answer, "
            "c.text AS context, collect(DISTINCT t.name) AS tags"
        )
        result = tx.run(query, subject_name=subject_name)
        return [record for record in result]

class ExamCreator:
    def __init__(self, db_connection):
        self.db = db_connection
        
    def create_or_get_subject(self, subject_name):
        return self.db.create_or_get_subject(subject_name)
        
    def add_question(self, subject_id, question_text, answer_text, context=None, tags=None):
        return self.db.create_question(subject_id, question_text, answer_text, context, tags)
    
    def generate_exam(self, subject_name):
        questions = self.db.get_exam_by_subject(subject_name)
        return {"subject": subject_name, "questions": questions}
    
# Example usage
def main():
    db = ExamDatabase("neo4j://localhost:7687", "neo4j", "akashgola")
    exam_creator = ExamCreator(db)
    subject_id = exam_creator.create_or_get_subject("Mathematics")
    
    q1_id = exam_creator.add_question(subject_id, "What is 2+2?", "4", "Basic arithmetic", ["math"])
    q2_id = exam_creator.add_question(subject_id, "What is 5x5?", "25", "Multiplication basics", ["math"])
    q3_id = exam_creator.add_question(subject_id, "What is the square root of 16?", "4", "Square roots", ["math"])
    
    db.link_all_questions(subject_id)
    
    exam = exam_creator.generate_exam("Mathematics")
    print(exam)
    db.close()

if __name__ == "__main__":
    main()



/tmp/ipykernel_6185/766083136.py:83: DeprecationWarning: write_transaction has been renamed to execute_write
  return session.write_transaction(self._create_or_get_subject_node, subject_name)
/tmp/ipykernel_6185/766083136.py:97: DeprecationWarning: write_transaction has been renamed to execute_write
  return session.write_transaction(


{'subject': 'Mathematics', 'questions': [<Record qid='c92f8dc7-688c-4ebe-8661-47452ad60993' question='What is the square root of 16?' answer='4' context='Square roots' tags=['math']>, <Record qid='cd32661c-aadd-4bd5-b381-96ed17d3c985' question='What is 5x5?' answer='25' context='Multiplication basics' tags=['math']>, <Record qid='5bc3c029-fca9-4060-89a8-a6ae7469fc12' question='What is 2+2?' answer='4' context='Basic arithmetic' tags=['math']>, <Record qid='9451b5b2-7d30-4701-a7da-d3e6ef836105' question='What is the formula for the circumference of a circle?' answer='C = 2πr' context='r represents the radius of the circle' tags=['geometry', 'formulas']>, <Record qid='e9291723-e794-42e8-864b-00cee9b7c296' question='Solve for x in the equation 2x + 5 = 13' answer='x = 4' context='Subtract 5 from both sides, then divide by 2' tags=['equations', 'algebra']>, <Record qid='74e9ef38-2ac6-40a7-b906-c262f87666d5' question='What is the formula for the area of a circle?' answer='A = πr²' context='

/tmp/ipykernel_6185/766083136.py:131: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._link_questions_in_subject, subject_id)
/tmp/ipykernel_6185/766083136.py:145: DeprecationWarning: read_transaction has been renamed to execute_read
  return session.read_transaction(self._get_questions_by_subject, subject_name)


In [31]:
import uuid
from AIAnswerEvaluationSystem.neo4j_Manager import Neo4jManager

class ExamDatabase:
    def __init__(self, uri, user, password):
        """
        Initializes the Neo4j connection.
        
        :param uri: Neo4j database URI
        :param user: Neo4j username
        :param password: Neo4j password
        """
        self.neo4j_manager = Neo4jManager.get_instance(uri, user, password)
        
    def close(self):
        self.driver.close()
        
    def create_or_get_subject(self, subject_name):
        with self.neo4j_manager.session() as session:
            return session.write_transaction(self._create_or_get_subject_node, subject_name)
            
    @staticmethod
    def _create_or_get_subject_node(tx, subject_name):
        query = (
            "MERGE (s:Subject {name: $subject_name}) "
            "ON CREATE SET s.id = $id "
            "RETURN s.id AS subject_id"
        )
        result = tx.run(query, subject_name=subject_name, id=str(uuid.uuid4()))
        return result.single()["subject_id"]
    
    def create_question_if_not_exists(self, subject_id, question_text, answer_text, context=None, tags=None):
        """Create a question only if it doesn't already exist with the same text"""
        with self.neo4j_manager.session() as session:
            return session.write_transaction(
                self._create_question_if_not_exists, subject_id, question_text, answer_text, context, tags)
            
    @staticmethod
    def _create_question_if_not_exists(tx, subject_id, question_text, answer_text, context, tags):
        # First check if the question already exists
        check_query = (
            "MATCH (s:Subject {id: $subject_id})-[:CONTAINS]->(q:Question {text: $question_text}) "
            "RETURN q.id AS question_id"
        )
        result = tx.run(check_query, subject_id=subject_id, question_text=question_text)
        record = result.single()
        
        # If question exists, return its ID
        if record:
            return record["question_id"]
        
        # Otherwise create new question
        question_id = str(uuid.uuid4())
        answer_id = str(uuid.uuid4())
        query = (
            "MATCH (s:Subject {id: $subject_id}) "
            "CREATE (q:Question {id: $question_id, text: $question_text})-[:HAS_ANSWER]->(a:Answer {id: $answer_id, text: $answer_text}) "
            "CREATE (s)-[:CONTAINS]->(q) "
        )
        params = {
            "subject_id": subject_id,
            "question_id": question_id,
            "question_text": question_text,
            "answer_id": answer_id,
            "answer_text": answer_text
        }
        if context:
            context_id = str(uuid.uuid4())
            query += "CREATE (q)-[:HAS_CONTEXT]->(c:Context {id: $context_id, text: $context_text}) "
            params.update({"context_id": context_id, "context_text": context})
        if tags:
            for i, tag in enumerate(tags):
                tag_param = f"tag_{i}"
                query += f"MERGE (t{i}:Tag {{name: ${tag_param}}}) CREATE (q)-[:TAGGED_WITH]->(t{i}) "
                params[tag_param] = tag
        query += "RETURN q.id AS question_id"
        result = tx.run(query, **params)
        return result.single()["question_id"]
    
    def link_all_questions(self, subject_id):
        with self.driver.session() as session:
            session.write_transaction(self._link_questions_in_subject, subject_id)
    
    @staticmethod
    def _link_questions_in_subject(tx, subject_id):
        # Modified to avoid creating duplicate relationships
        query = (
            "MATCH (s:Subject {id: $subject_id})-[:CONTAINS]->(q1:Question), "
            "(s)-[:CONTAINS]->(q2:Question) "
            "WHERE q1 <> q2 "
            "AND NOT EXISTS((q1)-[:RELATED_TO]->(q2)) "  # Only create relationship if it doesn't exist
            "MERGE (q1)-[:RELATED_TO]->(q2)"
        )
        tx.run(query, subject_id=subject_id)
    
    def get_exam_by_subject(self, subject_name):
        with self.driver.session() as session:
            return session.read_transaction(self._get_questions_by_subject, subject_name)
            
    @staticmethod
    def _get_questions_by_subject(tx, subject_name):
        query = (
            "MATCH (s:Subject {name: $subject_name})-[:CONTAINS]->(q:Question) "
            "MATCH (q)-[:HAS_ANSWER]->(a) "
            "OPTIONAL MATCH (q)-[:HAS_CONTEXT]->(c) "
            "OPTIONAL MATCH (q)-[:TAGGED_WITH]->(t) "
            "RETURN q.id AS qid, q.text AS question, a.text AS answer, "
            "c.text AS context, collect(DISTINCT t.name) AS tags"
        )
        result = tx.run(query, subject_name=subject_name)
        return [record for record in result]
    
    def count_questions_by_subject(self, subject_name):
        """Count how many questions exist for a given subject"""
        with self.driver.session() as session:
            return session.read_transaction(self._count_questions, subject_name)
    
    @staticmethod
    def _count_questions(tx, subject_name):
        query = (
            "MATCH (s:Subject {name: $subject_name})-[:CONTAINS]->(q:Question) "
            "RETURN count(q) AS question_count"
        )
        result = tx.run(query, subject_name=subject_name)
        return result.single()["question_count"]

class ExamCreator:
    def __init__(self, db_connection):
        self.db = db_connection
        
    def create_or_get_subject(self, subject_name):
        return self.db.create_or_get_subject(subject_name)
        
    def add_question(self, subject_id, question_text, answer_text, context=None, tags=None):
        return self.db.create_question_if_not_exists(subject_id, question_text, answer_text, context, tags)
    
    def generate_exam(self, subject_name):
        questions = self.db.get_exam_by_subject(subject_name)
        return {"subject": subject_name, "questions": questions}
    
    def get_question_count(self, subject_name):
        return self.db.count_questions_by_subject(subject_name)

# Example usage
def main():
    db = ExamDatabase("neo4j://localhost:7687", "neo4j", "akashgola")
    exam_creator = ExamCreator(db)
    
    # Create subjects
    subject_id_chemistry = exam_creator.create_or_get_subject("Chemistry")
    subject_id_physics = exam_creator.create_or_get_subject("Physics")
    
    # Print the current count of questions before adding new ones
    print(f"Chemistry questions before: {exam_creator.get_question_count('Chemistry')}")
    print(f"Physics questions before: {exam_creator.get_question_count('Physics')}")
    
    # Add questions to Chemistry
    exam_creator.add_question(subject_id_chemistry, "What are Atoms?", 
                          "The proton, neutron, and electron make up an atom, which is the smallest recognised division of a chemical element.", 
                          "Basic Chemistry", ["Chemistry"])
    
    # Add another chemistry question
    exam_creator.add_question(subject_id_chemistry, "What is the periodic table?", 
                          "The periodic table is a tabular arrangement of chemical elements organized by atomic number, electron configuration, and recurring chemical properties.", 
                          "Basic Chemistry", ["Chemistry", "Periodic Table"])
    
    # Add questions to Physics
    exam_creator.add_question(subject_id_physics, "What is Newton's second Law?", 
                          "Newton's second law of motion states that the acceleration of an object is directly proportional to the net force acting on it and inversely proportional to its mass.", 
                          "Newton's laws", ["Physics"])
    
    # Add another physics question
    exam_creator.add_question(subject_id_physics, "What is the law of conservation of energy?", 
                          "The law of conservation of energy states that energy can neither be created nor destroyed, only converted from one form to another.", 
                          "Energy", ["Physics", "Conservation Laws"])
    
    # Link all questions within each subject
    db.link_all_questions(subject_id_chemistry)
    db.link_all_questions(subject_id_physics)
    
    # Print the current count of questions after adding new ones
    print(f"Chemistry questions after: {exam_creator.get_question_count('Chemistry')}")
    print(f"Physics questions after: {exam_creator.get_question_count('Physics')}")
    
    # Generate and print exam for Chemistry
    exam_chemistry = exam_creator.generate_exam("Chemistry")
    print("\nChemistry Exam:")
    for i, q in enumerate(exam_chemistry["questions"], 1):
        print(f"Q{i}: {q['question']}")
        print(f"A: {q['answer']}")
        print(f"Context: {q['context']}")
        print(f"Tags: {q['tags']}")
        print()
    
    # Generate and print exam for Physics
    exam_physics = exam_creator.generate_exam("Physics")
    print("\nPhysics Exam:")
    for i, q in enumerate(exam_physics["questions"], 1):
        print(f"Q{i}: {q['question']}")
        print(f"A: {q['answer']}")
        print(f"Context: {q['context']}")
        print(f"Tags: {q['tags']}")
        print()
    
    db.close()

if __name__ == "__main__":
    main()

AttributeError: 'Neo4jManager' object has no attribute 'session'

Error executing write transaction: 'Neo4jManager' object has no attribute 'session'
Error executing write transaction: 'Neo4jManager' object has no attribute 'session'
Error executing read transaction: 'Neo4jManager' object has no attribute 'session'
Chemistry questions before: 0
Error executing read transaction: 'Neo4jManager' object has no attribute 'session'
Physics questions before: 0
Error executing write transaction: 'Neo4jManager' object has no attribute 'session'
Error executing write transaction: 'Neo4jManager' object has no attribute 'session'
Error executing write transaction: 'Neo4jManager' object has no attribute 'session'
Error executing write transaction: 'Neo4jManager' object has no attribute 'session'
Error executing write transaction: 'Neo4jManager' object has no attribute 'session'
Error executing write transaction: 'Neo4jManager' object has no attribute 'session'
Error executing read transaction: 'Neo4jManager' object has no attribute 'session'
Chemistry questions a

In [38]:
from neo4j import GraphDatabase
import uuid

class ExamDatabase:
    def __init__(self, uri, username, password):
        """
        Initializes the Neo4j connection.
        
        :param uri: Neo4j database URI
        :param username: Neo4j username
        :param password: Neo4j password
        """
        self.driver = GraphDatabase.driver(uri, auth=(username, password))
        
    def close(self):
        """Close the Neo4j connection"""
        self.driver.close()
        
    def _execute_read_transaction(self, func, *args):
        """
        Execute a read transaction using the Neo4j driver.
        """
        try:
            with self.driver.session() as session:
                return session.read_transaction(func, *args)
        except Exception as e:
            print(f"Error executing read transaction: {e}")
            return None

    def _execute_write_transaction(self, func, *args):
        """
        Execute a write transaction using the Neo4j driver.
        """
        try:
            with self.driver.session() as session:
                return session.write_transaction(func, *args)
        except Exception as e:
            print(f"Error executing write transaction: {e}")
            return None
            
    def create_or_get_subject(self, subject_name):
        """
        Creates a subject if it doesn't exist, or returns the existing one.
        
        :param subject_name: Name of the subject
        :return: Subject ID
        """
        return self._execute_write_transaction(self._create_or_get_subject_node, subject_name)
            
    @staticmethod
    def _create_or_get_subject_node(tx, subject_name):
        """
        Transaction function to create or get a subject node.
        """
        query = (
            "MERGE (s:Subject {name: $subject_name}) "
            "ON CREATE SET s.id = $id "
            "RETURN s.id AS subject_id"
        )
        result = tx.run(query, subject_name=subject_name, id=str(uuid.uuid4()))
        return result.single()["subject_id"]
    
    def create_question_if_not_exists(self, subject_id, question_text, answer_text, context=None, tags=None):
        """
        Create a question only if it doesn't already exist with the same text.
        
        :param subject_id: ID of the subject
        :param question_text: Text of the question
        :param answer_text: Text of the answer
        :param context: Optional context for the question
        :param tags: Optional list of tags for the question
        :return: Question ID
        """
        return self._execute_write_transaction(
            self._create_question_if_not_exists, subject_id, question_text, answer_text, context, tags)
            
    @staticmethod
    def _create_question_if_not_exists(tx, subject_id, question_text, answer_text, context, tags):
        # First check if the question already exists
        check_query = (
            "MATCH (s:Subject {id: $subject_id})-[:CONTAINS]->(q:Question {text: $question_text}) "
            "RETURN q.id AS question_id"
        )
        result = tx.run(check_query, subject_id=subject_id, question_text=question_text)
        record = result.single()
        if record:
            return record["question_id"]
        
        # Otherwise create new question
        question_id = str(uuid.uuid4())
        answer_id = str(uuid.uuid4())
        query = (
            "MATCH (s:Subject {id: $subject_id}) "
            "CREATE (q:Question {id: $question_id, text: $question_text})-[:HAS_ANSWER]->(a:Answer {id: $answer_id, text: $answer_text}) "
            "CREATE (s)-[:CONTAINS]->(q) "
        )
        params = {
            "subject_id": subject_id,
            "question_id": question_id,
            "question_text": question_text,
            "answer_id": answer_id,
            "answer_text": answer_text
        }
        if context:
            context_id = str(uuid.uuid4())
            query += "CREATE (q)-[:HAS_CONTEXT]->(c:Context {id: $context_id, text: $context_text}) "
            params.update({"context_id": context_id, "context_text": context})
        if tags:
            for i, tag in enumerate(tags):
                tag_param = f"tag_{i}"
                query += f"MERGE (t{i}:Tag {{name: ${tag_param}}}) CREATE (q)-[:TAGGED_WITH]->(t{i}) "
                params[tag_param] = tag
        query += "RETURN q.id AS question_id"
        result = tx.run(query, **params)
        return result.single()["question_id"]
    
    def create_question(self, subject_id, question_text, answer_text, context=None, tags=None):
        """
        Creates a new question regardless of whether it already exists.
        Maintained for backward compatibility.
        
        :param subject_id: ID of the subject
        :param question_text: Text of the question
        :param answer_text: Text of the answer
        :param context: Optional context for the question
        :param tags: Optional list of tags for the question
        :return: Question ID
        """
        return self._execute_write_transaction(self._create_question_node, subject_id, question_text, answer_text, context, tags)
    
    @staticmethod
    def _create_question_node(tx, subject_id, question_text, answer_text, context, tags):
        question_id = str(uuid.uuid4())
        answer_id = str(uuid.uuid4())
        query = (
            "MATCH (s:Subject {id: $subject_id}) "
            "CREATE (q:Question {id: $question_id, text: $question_text})-[:HAS_ANSWER]->(a:Answer {id: $answer_id, text: $answer_text}) "
            "CREATE (s)-[:CONTAINS]->(q) "
        )
        params = {
            "subject_id": subject_id,
            "question_id": question_id,
            "question_text": question_text,
            "answer_id": answer_id,
            "answer_text": answer_text
        }
        if context:
            context_id = str(uuid.uuid4())
            query += "CREATE (q)-[:HAS_CONTEXT]->(c:Context {id: $context_id, text: $context_text}) "
            params.update({"context_id": context_id, "context_text": context})
        if tags:
            for i, tag in enumerate(tags):
                tag_param = f"tag_{i}"
                query += f"MERGE (t{i}:Tag {{name: ${tag_param}}}) CREATE (q)-[:TAGGED_WITH]->(t{i}) "
                params[tag_param] = tag
        query += "RETURN q.id AS question_id"
        result = tx.run(query, **params)
        return result.single()["question_id"]
    
    def link_all_questions(self, subject_id):
        """
        Links all questions within a subject to each other.
        
        :param subject_id: ID of the subject
        """
        self._execute_write_transaction(self._link_questions_in_subject, subject_id)
    
    @staticmethod
    def _link_questions_in_subject(tx, subject_id):
        query = (
            "MATCH (s:Subject {id: $subject_id})-[:CONTAINS]->(q1:Question) "
            "MATCH (s)-[:CONTAINS]->(q2:Question) "
            "WHERE q1 <> q2 AND q1.id < q2.id "  # Using ID comparison to avoid duplicates
            "AND NOT EXISTS((q1)-[:RELATED_TO]->(q2)) "
            "WITH q1, q2 LIMIT 1000 "  # Process in batches of 1000
            "CREATE (q1)-[:RELATED_TO]->(q2) "
            "CREATE (q2)-[:RELATED_TO]->(q1)"   # Creating bidirectional relationships
        )
        tx.run(query, subject_id=subject_id)
        return None
    
    def get_exam_by_subject(self, subject_name):
        """
        Gets all questions for a subject.
        
        :param subject_name: Name of the subject
        :return: List of question dictionaries
        """
        result = self._execute_read_transaction(self._get_questions_by_subject, subject_name)
        return result if result else []
            
    @staticmethod
    def _get_questions_by_subject(tx, subject_name):
        query = (
            "MATCH (s:Subject {name: $subject_name})-[:CONTAINS]->(q:Question) "
            "MATCH (q)-[:HAS_ANSWER]->(a) "
            "OPTIONAL MATCH (q)-[:HAS_CONTEXT]->(c) "
            "OPTIONAL MATCH (q)-[:TAGGED_WITH]->(t) "
            "RETURN q.id AS qid, q.text AS question, a.text AS answer, "
            "c.text AS context, collect(DISTINCT t.name) AS tags"
        )
        result = tx.run(query, subject_name=subject_name)
        return [record for record in result]
    
    def count_questions_by_subject(self, subject_name):
        """
        Count how many questions exist for a given subject.
        
        :param subject_name: Name of the subject
        :return: Number of questions for the subject
        """
        result = self._execute_read_transaction(self._count_questions, subject_name)
        return result if result else 0
    
    @staticmethod
    def _count_questions(tx, subject_name):
        query = (
            "MATCH (s:Subject {name: $subject_name})-[:CONTAINS]->(q:Question) "
            "RETURN count(q) AS question_count"
        )
        result = tx.run(query, subject_name=subject_name)
        return result.single()["question_count"]

    def get_questions_by_tag(self, tag_name):
        """
        Gets all questions with a specific tag.
        
        :param tag_name: Name of the tag
        :return: List of question dictionaries
        """
        result = self._execute_read_transaction(self._get_questions_by_tag, tag_name)
        return result if result else []
    
    @staticmethod
    def _get_questions_by_tag(tx, tag_name):
        query = (
            "MATCH (t:Tag {name: $tag_name})<-[:TAGGED_WITH]-(q:Question) "
            "MATCH (q)-[:HAS_ANSWER]->(a) "
            "OPTIONAL MATCH (q)-[:HAS_CONTEXT]->(c) "
            "OPTIONAL MATCH (s:Subject)-[:CONTAINS]->(q) "
            "RETURN q.id AS qid, q.text AS question, a.text AS answer, "
            "c.text AS context, s.name AS subject"
        )
        result = tx.run(query, tag_name=tag_name)
        return [record for record in result]
    
    def delete_question(self, question_id):
        """
        Deletes a question and all its relationships.
        
        :param question_id: ID of the question to delete
        :return: True if successful, False otherwise
        """
        result = self._execute_write_transaction(self._delete_question, question_id)
        return bool(result) if result is not None else False
    
    @staticmethod
    def _delete_question(tx, question_id):
        query = (
            "MATCH (q:Question {id: $question_id}) "
            "OPTIONAL MATCH (q)-[r]-() "
            "DELETE r "
            "WITH q "
            "MATCH (q)-[:HAS_ANSWER]->(a) "
            "OPTIONAL MATCH (q)-[:HAS_CONTEXT]->(c) "
            "DELETE q, a, c "
            "RETURN count(q) > 0 AS deleted"
        )
        result = tx.run(query, question_id=question_id)
        return result.single()["deleted"]

class ExamCreator:
    def __init__(self, db_connection):
        """
        Initializes the ExamCreator with a database connection.
        
        :param db_connection: Instance of ExamDatabase
        """
        self.db = db_connection
        
    def create_or_get_subject(self, subject_name):
        """
        Creates a subject if it doesn't exist, or returns the existing one.
        
        :param subject_name: Name of the subject
        :return: Subject ID
        """
        return self.db.create_or_get_subject(subject_name)
        
    def add_question(self, subject_id, question_text, answer_text, context=None, tags=None):
        """
        Adds a question to a subject.
        
        :param subject_id: ID of the subject
        :param question_text: Text of the question
        :param answer_text: Text of the answer
        :param context: Optional context for the question
        :param tags: Optional list of tags for the question
        :return: Question ID
        """
        return self.db.create_question_if_not_exists(subject_id, question_text, answer_text, context, tags)
    
    def generate_exam(self, subject_name):
        """
        Generates an exam for a subject.
        
        :param subject_name: Name of the subject
        :return: Dictionary with subject name and questions
        """
        questions = self.db.get_exam_by_subject(subject_name)
        return {"subject": subject_name, "questions": questions}
    
    def get_question_count(self, subject_name):
        """
        Gets the number of questions for a subject.
        
        :param subject_name: Name of the subject
        :return: Number of questions
        """
        return self.db.count_questions_by_subject(subject_name)
    
    def delete_question(self, question_id):
        """
        Deletes a question.
        
        :param question_id: ID of the question to delete
        :return: True if successful, False otherwise
        """
        return self.db.delete_question(question_id)
    
    def get_questions_by_tag(self, tag_name):
        """
        Gets all questions with a specific tag.
        
        :param tag_name: Name of the tag
        :return: List of question dictionaries
        """
        return self.db.get_questions_by_tag(tag_name)
    
    def create_custom_exam(self, subject_name, num_questions=None, specific_tags=None):
        """
        Creates a custom exam with specific criteria.
        
        :param subject_name: Name of the subject
        :param num_questions: Number of questions to include (None for all)
        :param specific_tags: List of tags to filter by (None for all)
        :return: Dictionary with subject name and questions
        """
        try:
            with self.db.driver.session() as session:
                # Build the query based on criteria
                query = (
                    "MATCH (s:Subject {name: $subject_name})-[:CONTAINS]->(q:Question) "
                    "MATCH (q)-[:HAS_ANSWER]->(a) "
                    "OPTIONAL MATCH (q)-[:HAS_CONTEXT]->(c) "
                )
                
                # Add tag filtering if specified
                if specific_tags:
                    query += "MATCH (q)-[:TAGGED_WITH]->(t:Tag) WHERE t.name IN $tags "
                
                # Complete the query
                query += (
                    "OPTIONAL MATCH (q)-[:TAGGED_WITH]->(t) "
                    "WITH q, a, c, collect(DISTINCT t.name) AS tags "
                )
                
                # Add randomization and limiting if num_questions is specified
                if num_questions:
                    query += "ORDER BY rand() LIMIT $limit "
                
                # Final return statement
                query += "RETURN q.id AS qid, q.text AS question, a.text AS answer, c.text AS context, tags"
                
                # Prepare parameters
                params = {"subject_name": subject_name}
                if specific_tags:
                    params["tags"] = specific_tags
                if num_questions:
                    params["limit"] = num_questions
                
                # Execute the query
                result = session.run(query, **params)
                questions = [record for record in result]
                
                return {"subject": subject_name, "questions": questions}
        except Exception as e:
            print(f"Error creating custom exam: {e}")
            return {"subject": subject_name, "questions": []}

# Example usage
def main():
    try:
        db = ExamDatabase("neo4j://localhost:7687", "neo4j", "akashgola")
        exam_creator = ExamCreator(db)
        
        # Create subjects
        subject_id_math = exam_creator.create_or_get_subject("Mathematics")
        subject_id_physics = exam_creator.create_or_get_subject("Physics")
        subject_id_chemistry = exam_creator.create_or_get_subject("Chemistry")
        
        # Print the current count of questions before adding new ones
        print(f"Mathematics questions before: {exam_creator.get_question_count('Mathematics')}")
        print(f"Physics questions before: {exam_creator.get_question_count('Physics')}")
        print(f"Chemistry questions before: {exam_creator.get_question_count('Chemistry')}")
        
        # Add questions to Mathematics
        exam_creator.add_question(subject_id_math, "What is 2+2?", "4", "Basic arithmetic", ["math"])
        exam_creator.add_question(subject_id_math, "What is 5x5?", "25", "Multiplication basics", ["math"])
        exam_creator.add_question(subject_id_math, "What is the square root of 16?", "4", "Square roots", ["math"])
        
        # Add questions to Physics
        exam_creator.add_question(subject_id_physics, "What is Newton's First Law?", 
                            "An object remains at rest unless acted upon by a force", 
                            "Newton's laws", ["physics"])
        
        exam_creator.add_question(subject_id_physics, "What is Newton's second Law?", 
                            "Newton's second law of motion states that the acceleration of an object is directly proportional to the net force acting on it and inversely proportional to its mass.", 
                            "Newton's laws", ["physics"])
        
        # Add questions to Chemistry
        exam_creator.add_question(subject_id_chemistry, "What are Atoms?", 
                            "The proton, neutron, and electron make up an atom, which is the smallest recognised division of a chemical element.", 
                            "Basic Chemistry", ["chemistry"])
        
        # Link all questions within each subject
        db.link_all_questions(subject_id_math)
        db.link_all_questions(subject_id_physics)
        db.link_all_questions(subject_id_chemistry)
        
        # Print the current count of questions after adding new ones
        print(f"Mathematics questions after: {exam_creator.get_question_count('Mathematics')}")
        print(f"Physics questions after: {exam_creator.get_question_count('Physics')}")
        print(f"Chemistry questions after: {exam_creator.get_question_count('Chemistry')}")
        
        # Generate and print exam for Mathematics
        exam_math = exam_creator.generate_exam("Mathematics")
        print("\nMathematics Exam:")
        for i, q in enumerate(exam_math["questions"], 1):
            print(f"Q{i}: {q['question']}")
            print(f"A: {q['answer']}")
            print(f"Context: {q['context']}")
            print(f"Tags: {q['tags']}")
            print()
        
        # Generate a custom exam with specific criteria
        custom_exam = exam_creator.create_custom_exam("Mathematics", num_questions=2)
        print("\nCustom Mathematics Exam (2 random questions):")
        for i, q in enumerate(custom_exam["questions"], 1):
            print(f"Q{i}: {q['question']}")
            print(f"A: {q['answer']}")
            print(f"Context: {q['context']}")
            print(f"Tags: {q['tags']}")
            print()
            
        db.close()
    except Exception as e:
        print(f"An error occurred in main: {e}")

if __name__ == "__main__":
    main()

/tmp/ipykernel_6185/3666555344.py:36: DeprecationWarning: write_transaction has been renamed to execute_write
  return session.write_transaction(func, *args)
/tmp/ipykernel_6185/3666555344.py:25: DeprecationWarning: read_transaction has been renamed to execute_read
  return session.read_transaction(func, *args)


Mathematics questions before: 9
Physics questions before: 5
Chemistry questions before: 5


/home/gola/anaconda3/envs/GRAPH_RAG/lib/python3.10/site-packages/neo4j/_sync/work/result.py:625: UserWarning: Expected a result with a single record, but found multiple.
  warn(


Mathematics questions after: 9
Physics questions after: 6
Chemistry questions after: 5

Mathematics Exam:
Q1: What is the square root of 16?
A: 4
Context: Square roots
Tags: ['math']

Q2: What is 5x5?
A: 25
Context: Multiplication basics
Tags: ['math']

Q3: What is 2+2?
A: 4
Context: Basic arithmetic
Tags: ['math']

Q4: What is the formula for the circumference of a circle?
A: C = 2πr
Context: r represents the radius of the circle
Tags: ['geometry', 'formulas']

Q5: Solve for x in the equation 2x + 5 = 13
A: x = 4
Context: Subtract 5 from both sides, then divide by 2
Tags: ['equations', 'algebra']

Q6: What is the formula for the area of a circle?
A: A = πr²
Context: r represents the radius of the circle
Tags: ['formulas', 'geometry']

Q7: What is the formula for the circumference of a circle?
A: C = 2πr
Context: r represents the radius of the circle
Tags: ['formulas', 'geometry']

Q8: Solve for x in the equation 2x + 5 = 13
A: x = 4
Context: Subtract 5 from both sides, then divide by 

In [50]:
from neo4j import GraphDatabase
import uuid

class ExamDatabase:
    def __init__(self, uri, username, password):
        """
        Initializes the Neo4j connection.
        
        :param uri: Neo4j database URI
        :param username: Neo4j username
        :param password: Neo4j password
        """
        self.driver = GraphDatabase.driver(uri, auth=(username, password))
        
    def close(self):
        """Close the Neo4j connection"""
        self.driver.close()
        
    def _execute_read_transaction(self, func, *args):
        """
        Execute a read transaction using the Neo4j driver.
        """
        try:
            with self.driver.session() as session:
                return session.read_transaction(func, *args)
        except Exception as e:
            print(f"Error executing read transaction: {e}")
            return None

    def _execute_write_transaction(self, func, *args):
        """
        Execute a write transaction using the Neo4j driver.
        """
        try:
            with self.driver.session() as session:
                return session.write_transaction(func, *args)
        except Exception as e:
            print(f"Error executing write transaction: {e}")
            return None
            
    def create_or_get_subject(self, subject_name):
        """
        Creates a subject if it doesn't exist, or returns the existing one.
        
        :param subject_name: Name of the subject
        :return: Subject ID and subject code (e.g., MATH, PHYS, CHEM)
        """
        return self._execute_write_transaction(self._create_or_get_subject_node, subject_name)
            
    @staticmethod
    def _create_or_get_subject_node(tx, subject_name):
        """
        Transaction function to create or get a subject node.
        Generate a code field for each subject (taking first 4 letters, e.g., MATH for Mathematics)
        """
        # Generate subject code based on the subject name (first 4 letters, uppercase)
        subject_code = subject_name[:4].upper()
        
        query = (
            "MERGE (s:Subject {name: $subject_name}) "
            "ON CREATE SET s.id = $id, s.code = $subject_code, s.next_question_number = 1 "
            "ON MATCH SET s.code = $subject_code "
            "RETURN s.id AS subject_id, s.code AS subject_code"
        )
        result = tx.run(query, subject_name=subject_name, id=str(uuid.uuid4()), subject_code=subject_code)
        record = result.single()
        return record["subject_id"], record["subject_code"]
    
    def _get_next_question_number(self, subject_id):
        """
        Get and increment the next question number for a subject.
        
        :param subject_id: ID of the subject
        :return: Next question number
        """
        return self._execute_write_transaction(self._increment_question_number, subject_id)
    
    @staticmethod
    def _increment_question_number(tx, subject_id):
        """
        Transaction function to get and increment the next question number.
        """
        query = (
            "MATCH (s:Subject {id: $subject_id}) "
            "WITH s, s.next_question_number AS current_number "
            "SET s.next_question_number = current_number + 1 "
            "RETURN current_number"
        )
        result = tx.run(query, subject_id=subject_id)
        return result.single()["current_number"]
    
    def create_question_if_not_exists(self, subject_info, question_text, answer_text, context=None, tags=None):
        """
        Create a question only if it doesn't already exist with the same text.
        
        :param subject_info: Tuple containing (subject_id, subject_code)
        :param question_text: Text of the question
        :param answer_text: Text of the answer
        :param context: Optional context for the question
        :param tags: Optional list of tags for the question
        :return: Question ID in format "SUBJ001_Q1"
        """
        subject_id, subject_code = subject_info
        return self._execute_write_transaction(
            self._create_question_if_not_exists, subject_id, subject_code, question_text, answer_text, context, tags)
            
    @staticmethod
    def _create_question_if_not_exists(tx, subject_id, subject_code, question_text, answer_text, context, tags):
        # First check if the question already exists
        check_query = (
            "MATCH (s:Subject {id: $subject_id})-[:CONTAINS]->(q:Question {text: $question_text}) "
            "RETURN q.id AS question_id"
        )
        result = tx.run(check_query, subject_id=subject_id, question_text=question_text)
        record = result.single()
        if record:
            return record["question_id"]
        
        # Get the next question number for this subject
        number_query = (
            "MATCH (s:Subject {id: $subject_id}) "
            "WITH s, s.next_question_number AS current_number "
            "SET s.next_question_number = current_number + 1 "
            "RETURN current_number"
        )
        number_result = tx.run(number_query, subject_id=subject_id)
        question_number = number_result.single()["current_number"]
        
        # Format the question ID as SUBJ001_Q1
        formatted_number = f"{question_number:03d}"
        question_id = f"{subject_code}{formatted_number}_Q{question_number}"
        answer_id = str(uuid.uuid4())
        
        # Create the question
        query = (
            "MATCH (s:Subject {id: $subject_id}) "
            "CREATE (q:Question {id: $question_id, text: $question_text, number: $question_number})"
            "-[:HAS_ANSWER]->(a:Answer {id: $answer_id, text: $answer_text}) "
            "CREATE (s)-[:CONTAINS]->(q) "
        )
        params = {
            "subject_id": subject_id,
            "question_id": question_id,
            "question_text": question_text,
            "question_number": question_number,
            "answer_id": answer_id,
            "answer_text": answer_text
        }
        if context:
            context_id = str(uuid.uuid4())
            query += "CREATE (q)-[:HAS_CONTEXT]->(c:Context {id: $context_id, text: $context_text}) "
            params.update({"context_id": context_id, "context_text": context})
        if tags:
            for i, tag in enumerate(tags):
                tag_param = f"tag_{i}"
                query += f"MERGE (t{i}:Tag {{name: ${tag_param}}}) CREATE (q)-[:TAGGED_WITH]->(t{i}) "
                params[tag_param] = tag
        query += "RETURN q.id AS question_id"
        result = tx.run(query, **params)
        return result.single()["question_id"]
    
    def create_question(self, subject_info, question_text, answer_text, context=None, tags=None):
        """
        Creates a new question regardless of whether it already exists.
        Maintained for backward compatibility.
        
        :param subject_info: Tuple containing (subject_id, subject_code)
        :param question_text: Text of the question
        :param answer_text: Text of the answer
        :param context: Optional context for the question
        :param tags: Optional list of tags for the question
        :return: Question ID in format "SUBJ001_Q1"
        """
        subject_id, subject_code = subject_info
        return self._execute_write_transaction(
            self._create_question_node, subject_id, subject_code, question_text, answer_text, context, tags)
    
    @staticmethod
    def _create_question_node(tx, subject_id, subject_code, question_text, answer_text, context, tags):
        # Get the next question number for this subject
        number_query = (
            "MATCH (s:Subject {id: $subject_id}) "
            "WITH s, s.next_question_number AS current_number "
            "SET s.next_question_number = current_number + 1 "
            "RETURN current_number"
        )
        number_result = tx.run(number_query, subject_id=subject_id)
        question_number = number_result.single()["current_number"]
        
        # Format the question ID as SUBJ001_Q1
        formatted_number = f"{question_number:03d}"
        question_id = f"{subject_code}{formatted_number}_Q{question_number}"
        answer_id = str(uuid.uuid4())
        
        # Create the question
        query = (
            "MATCH (s:Subject {id: $subject_id}) "
            "CREATE (q:Question {id: $question_id, text: $question_text, number: $question_number})"
            "-[:HAS_ANSWER]->(a:Answer {id: $answer_id, text: $answer_text}) "
            "CREATE (s)-[:CONTAINS]->(q) "
        )
        params = {
            "subject_id": subject_id,
            "question_id": question_id,
            "question_text": question_text,
            "question_number": question_number,
            "answer_id": answer_id,
            "answer_text": answer_text
        }
        if context:
            context_id = str(uuid.uuid4())
            query += "CREATE (q)-[:HAS_CONTEXT]->(c:Context {id: $context_id, text: $context_text}) "
            params.update({"context_id": context_id, "context_text": context})
        if tags:
            for i, tag in enumerate(tags):
                tag_param = f"tag_{i}"
                query += f"MERGE (t{i}:Tag {{name: ${tag_param}}}) CREATE (q)-[:TAGGED_WITH]->(t{i}) "
                params[tag_param] = tag
        query += "RETURN q.id AS question_id"
        result = tx.run(query, **params)
        return result.single()["question_id"]
    
    def link_all_questions(self, subject_id):
        """
        Links all questions within a subject to each other.
        
        :param subject_id: ID of the subject
        """
        self._execute_write_transaction(self._link_questions_in_subject, subject_id)
    
    @staticmethod
    def _link_questions_in_subject(tx, subject_id):
        query = (
            "MATCH (s:Subject {id: $subject_id})-[:CONTAINS]->(q1:Question) "
            "MATCH (s)-[:CONTAINS]->(q2:Question) "
            "WHERE q1 <> q2 AND q1.id < q2.id "  # Using ID comparison to avoid duplicates
            "AND NOT EXISTS((q1)-[:RELATED_TO]->(q2)) "
            "WITH q1, q2 LIMIT 1000 "  # Process in batches of 1000
            "CREATE (q1)-[:RELATED_TO]->(q2) "
            "CREATE (q2)-[:RELATED_TO]->(q1)"   # Creating bidirectional relationships
        )
        tx.run(query, subject_id=subject_id)
        return None
    
    def get_exam_by_subject(self, subject_name):
        """
        Gets all questions for a subject.
        
        :param subject_name: Name of the subject
        :return: List of question dictionaries
        """
        result = self._execute_read_transaction(self._get_questions_by_subject, subject_name)
        return result if result else []
            
    @staticmethod
    def _get_questions_by_subject(tx, subject_name):
        # Fixed query with proper handling of aggregation
        query = (
            "MATCH (s:Subject {name: $subject_name})-[:CONTAINS]->(q:Question) "
            "MATCH (q)-[:HAS_ANSWER]->(a) "
            "OPTIONAL MATCH (q)-[:HAS_CONTEXT]->(c) "
            "OPTIONAL MATCH (q)-[:TAGGED_WITH]->(t) "
            "WITH q, a, c, collect(DISTINCT t.name) AS tags "  # Group by q, a, c first
            "ORDER BY q.number "  # Then order
            "RETURN q.id AS qid, q.text AS question, a.text AS answer, "
            "c.text AS context, tags"  # Return the collected tags
        )
        result = tx.run(query, subject_name=subject_name)
        return [record for record in result]
    
    def count_questions_by_subject(self, subject_name):
        """
        Count how many questions exist for a given subject.
        
        :param subject_name: Name of the subject
        :return: Number of questions for the subject
        """
        result = self._execute_read_transaction(self._count_questions, subject_name)
        return result if result else 0
    
    @staticmethod
    def _count_questions(tx, subject_name):
        query = (
            "MATCH (s:Subject {name: $subject_name})-[:CONTAINS]->(q:Question) "
            "RETURN count(q) AS question_count"
        )
        result = tx.run(query, subject_name=subject_name)
        return result.single()["question_count"]

    def get_questions_by_tag(self, tag_name):
        """
        Gets all questions with a specific tag.
        
        :param tag_name: Name of the tag
        :return: List of question dictionaries
        """
        result = self._execute_read_transaction(self._get_questions_by_tag, tag_name)
        return result if result else []
    
    @staticmethod
    def _get_questions_by_tag(tx, tag_name):
        # Fixed query with proper handling of ordering after collecting data
        query = (
            "MATCH (t:Tag {name: $tag_name})<-[:TAGGED_WITH]-(q:Question) "
            "MATCH (q)-[:HAS_ANSWER]->(a) "
            "OPTIONAL MATCH (q)-[:HAS_CONTEXT]->(c) "
            "OPTIONAL MATCH (s:Subject)-[:CONTAINS]->(q) "
            "WITH q, a, c, s "  # Group these first
            "ORDER BY q.number "  # Then order
            "RETURN q.id AS qid, q.text AS question, a.text AS answer, "
            "c.text AS context, s.name AS subject"
        )
        result = tx.run(query, tag_name=tag_name)
        return [record for record in result]
    
    def delete_question(self, question_id):
        """
        Deletes a question and all its relationships.
        
        :param question_id: ID of the question to delete
        :return: True if successful, False otherwise
        """
        result = self._execute_write_transaction(self._delete_question, question_id)
        return bool(result) if result is not None else False
    
    @staticmethod
    def _delete_question(tx, question_id):
        query = (
            "MATCH (q:Question {id: $question_id}) "
            "OPTIONAL MATCH (q)-[r]-() "
            "DELETE r "
            "WITH q "
            "MATCH (q)-[:HAS_ANSWER]->(a) "
            "OPTIONAL MATCH (q)-[:HAS_CONTEXT]->(c) "
            "DELETE q, a, c "
            "RETURN count(q) > 0 AS deleted"
        )
        result = tx.run(query, question_id=question_id)
        return result.single()["deleted"]

class ExamCreator:
    def __init__(self, db_connection):
        """
        Initializes the ExamCreator with a database connection.
        
        :param db_connection: Instance of ExamDatabase
        """
        self.db = db_connection
        
    def create_or_get_subject(self, subject_name):
        """
        Creates a subject if it doesn't exist, or returns the existing one.
        
        :param subject_name: Name of the subject
        :return: Subject info (id, code)
        """
        return self.db.create_or_get_subject(subject_name)
        
    def add_question(self, subject_info, question_text, answer_text, context=None, tags=None):
        """
        Adds a question to a subject, ensuring consistent ID format "SUBJ001_Q1".
        
        :param subject_info: Tuple containing (subject_id, subject_code)
        :param question_text: Text of the question
        :param answer_text: Text of the answer
        :param context: Optional context for the question
        :param tags: Optional list of tags for the question
        :return: Question ID
        """
        return self.db.create_question_if_not_exists(subject_info, question_text, answer_text, context, tags)
    
    def generate_exam(self, subject_name):
        """
        Generates an exam for a subject.
        
        :param subject_name: Name of the subject
        :return: Dictionary with subject name and questions
        """
        questions = self.db.get_exam_by_subject(subject_name)
        return {"subject": subject_name, "questions": questions}
    
    def get_question_count(self, subject_name):
        """
        Gets the number of questions for a subject.
        
        :param subject_name: Name of the subject
        :return: Number of questions
        """
        return self.db.count_questions_by_subject(subject_name)
    
    def delete_question(self, question_id):
        """
        Deletes a question.
        
        :param question_id: ID of the question to delete
        :return: True if successful, False otherwise
        """
        return self.db.delete_question(question_id)
    
    def get_questions_by_tag(self, tag_name):
        """
        Gets all questions with a specific tag.
        
        :param tag_name: Name of the tag
        :return: List of question dictionaries
        """
        return self.db.get_questions_by_tag(tag_name)
    
    def create_custom_exam(self, subject_name, num_questions=None, specific_tags=None):
        """
        Creates a custom exam with specific criteria.
        
        :param subject_name: Name of the subject
        :param num_questions: Number of questions to include (None for all)
        :param specific_tags: List of tags to filter by (None for all)
        :return: Dictionary with subject name and questions
        """
        try:
            with self.db.driver.session() as session:
                # Build the query based on criteria
                query = (
                    "MATCH (s:Subject {name: $subject_name})-[:CONTAINS]->(q:Question) "
                    "MATCH (q)-[:HAS_ANSWER]->(a) "
                    "OPTIONAL MATCH (q)-[:HAS_CONTEXT]->(c) "
                )
                
                # Add tag filtering if specified
                if specific_tags:
                    query += "MATCH (q)-[:TAGGED_WITH]->(t:Tag) WHERE t.name IN $tags "
                
                # Complete the query with proper handling of tags collection
                query += (
                    "OPTIONAL MATCH (q)-[:TAGGED_WITH]->(t) "
                    "WITH q, a, c, collect(DISTINCT t.name) AS tags "
                )
                
                # Add randomization and limiting if num_questions is specified
                if num_questions:
                    query += "ORDER BY rand() LIMIT $limit "
                else:
                    query += "ORDER BY q.number "  # Order by question number if not random
                
                # Final return statement
                query += "RETURN q.id AS qid, q.text AS question, a.text AS answer, c.text AS context, tags"
                
                # Prepare parameters
                params = {"subject_name": subject_name}
                if specific_tags:
                    params["tags"] = specific_tags
                if num_questions:
                    params["limit"] = num_questions
                
                # Execute the query
                result = session.run(query, **params)
                questions = [record for record in result]
                
                return {"subject": subject_name, "questions": questions}
        except Exception as e:
            print(f"Error creating custom exam: {e}")
            return {"subject": subject_name, "questions": []}

# Example usage
def main():
    try:
        # Connect to Neo4j
        db = ExamDatabase("neo4j://localhost:7687", "neo4j", "your_password")
        exam_creator = ExamCreator(db)
        
        # Create subjects with consistent 4-letter codes
        subject_info_math = exam_creator.create_or_get_subject("Mathematics")       # Code: MATH
        subject_info_physics = exam_creator.create_or_get_subject("Physics")        # Code: PHYS
        subject_info_chemistry = exam_creator.create_or_get_subject("Chemistry")    # Code: CHEM
        subject_info_ai = exam_creator.create_or_get_subject("Artificial Intelligence")  # Code: ARTI
        subject_info_hindi = exam_creator.create_or_get_subject("Hindi")           # Code: HIND
        
        # Print the current count of questions before adding new ones
        print(f"Mathematics questions before: {exam_creator.get_question_count('Mathematics')}")
        print(f"Physics questions before: {exam_creator.get_question_count('Physics')}")
        print(f"Chemistry questions before: {exam_creator.get_question_count('Chemistry')}")
        print(f"AI questions before: {exam_creator.get_question_count('Artificial Intelligence')}")
        print(f"Hindi questions before: {exam_creator.get_question_count('Hindi')}")
        
        # Add questions to Mathematics
        q1_id = exam_creator.add_question(subject_info_math, "What is the formula for the circumference of a circle?", 
                                         "2πr", "Circle geometry", ["geometry", "math"])
        
        q2_id = exam_creator.add_question(subject_info_math, "Solve for x in the equation 2x + 13", 
                                         "x = -6.5", "Basic algebra", ["algebra", "math"])
                                         
        q3_id = exam_creator.add_question(subject_info_math, "Solve x² - 5x + 6 = 0", 
                                         "x = 2 or x = 3", "Quadratic equations", ["algebra", "math"])
        
        # Add questions to Physics
        q4_id = exam_creator.add_question(subject_info_physics, "Explain Newton's First Law", 
                            "An object at rest stays at rest and an object in motion stays in motion with the same speed and in the same direction unless acted upon by an external force.", 
                            "Newton's laws", ["mechanics", "physics"])
        
        # Add questions to Chemistry
        q5_id = exam_creator.add_question(subject_info_chemistry, "Balance Na + H₂O reaction", 
                            "2Na + 2H₂O → 2NaOH + H₂", 
                            "Chemical reactions", ["reactions", "chemistry"])
        
        # Add questions to AI
        q6_id = exam_creator.add_question(subject_info_ai, "Define Artificial Intelligence", 
                            "Artificial Intelligence (AI) is the simulation of human intelligence processes by machines, especially computer systems.", 
                            "AI basics", ["ai", "computer science"])
                            
        # Add questions to Hindi
        q7_id = exam_creator.add_question(subject_info_hindi, "संस्कृत भाषा का महत्व क्या है?", 
                            "संस्कृत भारत की प्राचीनतम भाषाओं में से एक है और इसे 'देववाणी' भी कहा जाता है।", 
                            "भाषा", ["संस्कृत", "भाषा"])
        
        # Link all questions within each subject
        db.link_all_questions(subject_info_math[0])
        db.link_all_questions(subject_info_physics[0])
        db.link_all_questions(subject_info_chemistry[0])
        db.link_all_questions(subject_info_ai[0])
        db.link_all_questions(subject_info_hindi[0])
        
        # Print the question IDs to verify format
        print(f"\nQuestion IDs:")
        print(f"Mathematics Q1: {q1_id}")  # Should be MATH001_Q1
        print(f"Mathematics Q2: {q2_id}")  # Should be MATH002_Q1
        print(f"Mathematics Q3: {q3_id}")  # Should be MATH003_Q1
        print(f"Physics Q1: {q4_id}")      # Should be PHYS001_Q1
        print(f"Chemistry Q1: {q5_id}")    # Should be CHEM001_Q1
        print(f"AI Q1: {q6_id}")           # Should be ARTI001_Q1
        print(f"Hindi Q1: {q7_id}")        # Should be HIND001_Q1
        
        # Print the current count of questions after adding new ones
        print(f"\nMathematics questions after: {exam_creator.get_question_count('Mathematics')}")
        print(f"Physics questions after: {exam_creator.get_question_count('Physics')}")
        print(f"Chemistry questions after: {exam_creator.get_question_count('Chemistry')}")
        print(f"AI questions after: {exam_creator.get_question_count('Artificial Intelligence')}")
        print(f"Hindi questions after: {exam_creator.get_question_count('Hindi')}")
        
        # Generate and print exam for Mathematics
        exam_math = exam_creator.generate_exam("Mathematics")
        print("\nMathematics Exam:")
        for i, q in enumerate(exam_math["questions"], 1):
            print(f"ID: {q['qid']}")
            print(f"Q: {q['question']}")
            print(f"A: {q['answer']}")
            print(f"Context: {q['context']}")
            print(f"Tags: {q['tags']}")
            print()
        
        # Generate a custom exam with specific criteria
        custom_exam = exam_creator.create_custom_exam("Mathematics", num_questions=2)
        print("\nCustom Mathematics Exam (2 random questions):")
        for i, q in enumerate(custom_exam["questions"], 1):
            print(f"ID: {q['qid']}")
            print(f"Q: {q['question']}")
            print(f"A: {q['answer']}")
            print(f"Context: {q['context']}")
            print(f"Tags: {q['tags']}")
            print()
            
        db.close()
    except Exception as e:
        print(f"An error occurred in main: {e}")

if __name__ == "__main__":
    main()

/tmp/ipykernel_6185/508905744.py:36: DeprecationWarning: write_transaction has been renamed to execute_write
  return session.write_transaction(func, *args)
/tmp/ipykernel_6185/508905744.py:25: DeprecationWarning: read_transaction has been renamed to execute_read
  return session.read_transaction(func, *args)


Mathematics questions before: 9
Physics questions before: 6
Chemistry questions before: 5
AI questions before: 0
Hindi questions before: 0
Error executing write transaction: unsupported format string passed to NoneType.__format__
Error executing write transaction: unsupported format string passed to NoneType.__format__
Error executing write transaction: unsupported format string passed to NoneType.__format__
Error executing write transaction: unsupported format string passed to NoneType.__format__
Error executing write transaction: unsupported format string passed to NoneType.__format__
Error executing write transaction: unsupported format string passed to NoneType.__format__

Question IDs:
Mathematics Q1: 9451b5b2-7d30-4701-a7da-d3e6ef836105
Mathematics Q2: None
Mathematics Q3: None
Physics Q1: None
Chemistry Q1: None
AI Q1: None
Hindi Q1: None

Mathematics questions after: 9
Physics questions after: 6
Chemistry questions after: 5
AI questions after: 0
Hindi questions after: 0

Mathem